In [1669]:
#  Allows charts to appear in the notebook
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Libraries for analysis
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

# Libraries for visuals
import matplotlib.pyplot as plt
import seaborn as sns

# allows all columns in the dataframe to be seen rather than '...' in the middle section
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)


In [1670]:
# reading in the txt file
data=pd.read_table('AviationData.txt')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  


In [1671]:
# checking what the file looks like
data.head()

,Event Id | Investigation Type | Accident Number | Event Date | Location | Country | Latitude | Longitude | Airport Code | Airport Name | Injury Severity | Aircraft Damage | Aircraft Category | Registration Number | Make | Model | Amateur Built | Number of Engines | Engine Type | FAR Description | Schedule | Purpose of Flight | Air Carrier | Total Fatal Injuries | Total Serious Injuries | Total Minor Injuries | Total Uninjured | Weather Condition | Broad Phase of Flight | Report Status | Publication Date |
0,"20190416X94734 | Accident | WPR19LA111 | 04/16/2019 | Hauula, HI | United States | 21.588333 | -157.911944 | | N/A | Non-Fatal | Substantial | Helicopter | N593C | MD HELICOPTER | 369 | No | 1 | Turbo Shaft | Part 135: Air Taxi & Commuter | NSCH | | K & S Helicopters Inc | | | | 4 | VMC | CRUISE | Preliminary | 04/18/2019 |"
1,"20190411X54423 | Accident | GAA19CA203 | 04/11/2019 | Fullteron, CA | United States | 33.871945 | -117.979723 | FUL | Fullerton Muni | Unavailable | Substantial | | N9964L | Cessna | 172 | No | 1 | | Part 91: General Aviation | | Instructional | | | | | | VMC | | Preliminary | 04/20/2019 |"
2,"20190411X15518 | Accident | ERA19LA149 | 04/11/2019 | Meriden, CT | United States | 41.526945 | -72.831389 | MMK | Meriden Markham Muni | Non-Fatal | Substantial | Airplane | N38658 | Piper | PA28R | No | 1 | Reciprocating | Part 91: General Aviation | | Personal | | | 2 | | | VMC | APPROACH | Preliminary | 04/19/2019 |"
3,"20190409X51503 | Accident | CEN19FA117 | 04/09/2019 | Harlingen, TX | United States | 26.292500 | -97.765000 | | N/A | Fatal(1) | Destroyed | Airplane | N7314D | Air Tractor | AT 502 | No | 1 | Turbo Prop | Part 137: Agricultural | | Aerial Application | | 1 | | | | VMC | MANEUVERING | Preliminary | 04/18/2019 |"
4,"20190408X12655 | Accident | GAA19CA200 | 04/04/2019 | Fort Worth, TX | United States | 32.931111 | -97.411667 | T67 | HICKS AIRFIELD | Non-Fatal | Substantial | Airplane | N801R | Zenair | ZENITH CH 801 | Yes | 1 | | Part 91: General Aviation | | Personal | | | | | 3 | VMC | | Preliminary | 04/10/2019 |"


In [1672]:
# splitting table on pipe characters
data=pd.read_table('AviationData.txt', delimiter='|')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead.
  


In [1673]:
# checking what the dataframe looks like now
data.head()

,Event Id,Investigation Type,Accident Number,Event Date,Location,Country,Latitude,Longitude,Airport Code,Airport Name,Injury Severity,Aircraft Damage,Aircraft Category,Registration Number,Make,Model,Amateur Built,Number of Engines,Engine Type,FAR Description,Schedule,Purpose of Flight,Air Carrier,Total Fatal Injuries,Total Serious Injuries,Total Minor Injuries,Total Uninjured,Weather Condition,Broad Phase of Flight,Report Status,Publication Date,
0,20190416X94734,Accident,WPR19LA111,04/16/2019,"Hauula, HI",United States,21.588333,-157.911944,,N/A,Non-Fatal,Substantial,Helicopter,N593C,MD HELICOPTER,369,No,1,Turbo Shaft,Part 135: Air Taxi & Commuter,NSCH,,K & S Helicopters Inc,,,,4,VMC,CRUISE,Preliminary,04/18/2019,
1,20190411X54423,Accident,GAA19CA203,04/11/2019,"Fullteron, CA",United States,33.871945,-117.979723,FUL,Fullerton Muni,Unavailable,Substantial,,N9964L,Cessna,172,No,1,,Part 91: General Aviation,,Instructional,,,,,,VMC,,Preliminary,04/20/2019,
2,20190411X15518,Accident,ERA19LA149,04/11/2019,"Meriden, CT",United States,41.526945,-72.831389,MMK,Meriden Markham Muni,Non-Fatal,Substantial,Airplane,N38658,Piper,PA28R,No,1,Reciprocating,Part 91: General Aviation,,Personal,,,2,,,VMC,APPROACH,Preliminary,04/19/2019,
3,20190409X51503,Accident,CEN19FA117,04/09/2019,"Harlingen, TX",United States,26.292500,-97.765000,,N/A,Fatal(1),Destroyed,Airplane,N7314D,Air Tractor,AT 502,No,1,Turbo Prop,Part 137: Agricultural,,Aerial Application,,1,,,,VMC,MANEUVERING,Preliminary,04/18/2019,
4,20190408X12655,Accident,GAA19CA200,04/04/2019,"Fort Worth, TX",United States,32.931111,-97.411667,T67,HICKS AIRFIELD,Non-Fatal,Substantial,Airplane,N801R,Zenair,ZENITH CH 801,Yes,1,,Part 91: General Aviation,,Personal,,,,,3,VMC,,Preliminary,04/10/2019,


In [1674]:
# checking the size of the dataframe
data.shape

(82977, 32)

In [1675]:
# looking to see what is in an "empty cell" in the first row of the airport code column
a=data.iloc[0:1, 8:9]
a

,Airport Code
0,


In [1676]:
# trying to find out more about what is in the empty cell
a.info

<bound method DataFrame.info of    Airport Code 
0               >

In [1677]:
# checking results against a cell where I can see a value
b=data.iloc[0:1, 1:2]
b

,Investigation Type
0,Accident


In [1678]:
b.info

<bound method DataFrame.info of    Investigation Type 
0   Accident          >

In [1679]:
# checking how the dataset sees the empty cells. It doesn't seem to identify them as null values so there must be
# something in the cell. Also note that the Event ID is not aligned to other column names.
data.isna().sum()

Event Id                    0
 Investigation Type         0
 Accident Number            0
 Event Date                 0
 Location                   0
 Country                    0
 Latitude                   0
 Longitude                  0
 Airport Code               0
 Airport Name               0
 Injury Severity            0
 Aircraft Damage            0
 Aircraft Category          0
 Registration Number        0
 Make                       0
 Model                      0
 Amateur Built              0
 Number of Engines          0
 Engine Type                0
 FAR Description            0
 Schedule                   0
 Purpose of Flight          0
 Air Carrier                0
 Total Fatal Injuries       0
 Total Serious Injuries     0
 Total Minor Injuries       0
 Total Uninjured            0
 Weather Condition          0
 Broad Phase of Flight      0
 Report Status              0
 Publication Date           0
                            0
dtype: int64

In [1680]:
# Looking at the columns, I can see something a little odd with the formatting of the column names, with a white space 
# between the quote marks and column names
data.columns

Index(['Event Id ', ' Investigation Type ', ' Accident Number ', ' Event Date ', ' Location ', ' Country ', ' Latitude ', ' Longitude ', ' Airport Code ', ' Airport Name ', ' Injury Severity ', ' Aircraft Damage ', ' Aircraft Category ', ' Registration Number ', ' Make ', ' Model ', ' Amateur Built ', ' Number of Engines ', ' Engine Type ', ' FAR Description ', ' Schedule ', ' Purpose of Flight ', ' Air Carrier ', ' Total Fatal Injuries ', ' Total Serious Injuries ', ' Total Minor Injuries ', ' Total Uninjured ', ' Weather Condition ', ' Broad Phase of Flight ', ' Report Status ', ' Publication Date ', ' '], dtype='object')

In [1681]:
# making a copy in order to keep the original dataframe before changes are made
cleaning1=data.copy()

In [1682]:
cleaning1.head(2)

,Event Id,Investigation Type,Accident Number,Event Date,Location,Country,Latitude,Longitude,Airport Code,Airport Name,Injury Severity,Aircraft Damage,Aircraft Category,Registration Number,Make,Model,Amateur Built,Number of Engines,Engine Type,FAR Description,Schedule,Purpose of Flight,Air Carrier,Total Fatal Injuries,Total Serious Injuries,Total Minor Injuries,Total Uninjured,Weather Condition,Broad Phase of Flight,Report Status,Publication Date,
0,20190416X94734,Accident,WPR19LA111,04/16/2019,"Hauula, HI",United States,21.588333,-157.911944,,N/A,Non-Fatal,Substantial,Helicopter,N593C,MD HELICOPTER,369,No,1,Turbo Shaft,Part 135: Air Taxi & Commuter,NSCH,,K & S Helicopters Inc,,,,4,VMC,CRUISE,Preliminary,04/18/2019,
1,20190411X54423,Accident,GAA19CA203,04/11/2019,"Fullteron, CA",United States,33.871945,-117.979723,FUL,Fullerton Muni,Unavailable,Substantial,,N9964L,Cessna,172,No,1,,Part 91: General Aviation,,Instructional,,,,,,VMC,,Preliminary,04/20/2019,


In [1683]:
# cleaning the column names
# removing white space, making changing to lowercase, and put '_' between words.
cleaning1.columns=[col.strip().replace(' ','_').lower() for col in cleaning1.columns]


In [1684]:
# checking how they appear now
cleaning1.columns

Index(['event_id', 'investigation_type', 'accident_number', 'event_date', 'location', 'country', 'latitude', 'longitude', 'airport_code', 'airport_name', 'injury_severity', 'aircraft_damage', 'aircraft_category', 'registration_number', 'make', 'model', 'amateur_built', 'number_of_engines', 'engine_type', 'far_description', 'schedule', 'purpose_of_flight', 'air_carrier', 'total_fatal_injuries', 'total_serious_injuries', 'total_minor_injuries', 'total_uninjured', 'weather_condition', 'broad_phase_of_flight', 'report_status', 'publication_date', ''], dtype='object')

In [1685]:
# Now that column names are sorted, checking if the empty cell in the Airport Code
# column is an empty string using a single space between the string marks. 
#  Not identified.

(cleaning1['airport_code'] == ' ')

0        False
1        False
2        False
3        False
4        False
5        False
6        False
7        False
8        False
9        False
10       False
11       False
12       False
13       False
14       False
15       False
16       False
17       False
18       False
19       False
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27       False
28       False
29       False
30       False
31       False
32       False
33       False
34       False
35       False
36       False
37       False
38       False
39       False
40       False
41       False
42       False
43       False
44       False
45       False
46       False
47       False
48       False
49       False
50       False
51       False
52       False
53       False
54       False
55       False
56       False
57       False
58       False
59       False
60       False
61       False
62       False
63       False
64       False
65       False
66       F

In [1686]:
# Trying empty strings with two spaces. Found what is in the empty cell.
(cleaning1['airport_code'] == '  ')

0        True 
1        False
2        False
3        True 
4        False
5        False
6        True 
7        True 
8        False
9        False
10       False
11       False
12       True 
13       False
14       True 
15       False
16       False
17       False
18       True 
19       True 
20       True 
21       True 
22       False
23       True 
24       False
25       False
26       False
27       True 
28       False
29       False
30       False
31       False
32       False
33       False
34       True 
35       False
36       True 
37       False
38       False
39       False
40       False
41       False
42       False
43       False
44       True 
45       False
46       False
47       True 
48       False
49       False
50       False
51       False
52       True 
53       False
54       False
55       False
56       False
57       False
58       False
59       False
60       True 
61       False
62       False
63       False
64       False
65       True 
66       F

In [1687]:
cleaning1.head(2)

,event_id,investigation_type,accident_number,event_date,location,country,latitude,longitude,airport_code,airport_name,injury_severity,aircraft_damage,aircraft_category,registration_number,make,model,amateur_built,number_of_engines,engine_type,far_description,schedule,purpose_of_flight,air_carrier,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight,report_status,publication_date,
0,20190416X94734,Accident,WPR19LA111,04/16/2019,"Hauula, HI",United States,21.588333,-157.911944,,N/A,Non-Fatal,Substantial,Helicopter,N593C,MD HELICOPTER,369,No,1,Turbo Shaft,Part 135: Air Taxi & Commuter,NSCH,,K & S Helicopters Inc,,,,4,VMC,CRUISE,Preliminary,04/18/2019,
1,20190411X54423,Accident,GAA19CA203,04/11/2019,"Fullteron, CA",United States,33.871945,-117.979723,FUL,Fullerton Muni,Unavailable,Substantial,,N9964L,Cessna,172,No,1,,Part 91: General Aviation,,Instructional,,,,,,VMC,,Preliminary,04/20/2019,


In [1688]:
# changing empty strings to to np.nan so NaNs can be counted
cleaning1.replace({'  ': np.nan}, inplace = True)

In [1689]:
# checking it has worked by looking at the first row and airport code column.
cleaning1.head(2)

,event_id,investigation_type,accident_number,event_date,location,country,latitude,longitude,airport_code,airport_name,injury_severity,aircraft_damage,aircraft_category,registration_number,make,model,amateur_built,number_of_engines,engine_type,far_description,schedule,purpose_of_flight,air_carrier,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight,report_status,publication_date,
0,20190416X94734,Accident,WPR19LA111,04/16/2019,"Hauula, HI",United States,21.588333,-157.911944,NaN,N/A,Non-Fatal,Substantial,Helicopter,N593C,MD HELICOPTER,369,No,1,Turbo Shaft,Part 135: Air Taxi & Commuter,NSCH,NaN,K & S Helicopters Inc,NaN,NaN,NaN,4,VMC,CRUISE,Preliminary,04/18/2019,
1,20190411X54423,Accident,GAA19CA203,04/11/2019,"Fullteron, CA",United States,33.871945,-117.979723,FUL,Fullerton Muni,Unavailable,Substantial,NaN,N9964L,Cessna,172,No,1,NaN,Part 91: General Aviation,NaN,Instructional,NaN,NaN,NaN,NaN,NaN,VMC,NaN,Preliminary,04/20/2019,


In [1690]:
# Now the NaNs are visible, checking how many there are in the dataframe. 
# My approach will be to remove unneeded columns before dealing with NaNs
cleaning1.isna().sum()

event_id                  0    
investigation_type        3    
accident_number           0    
event_date                0    
location                  77   
country                   507  
latitude                  53929
longitude                 53938
airport_code              35861
airport_name              30462
injury_severity           0    
aircraft_damage           2622 
aircraft_category         56737
registration_number       3633 
make                      71   
model                     99   
amateur_built             635  
number_of_engines         4670 
engine_type               4004 
far_description           57070
schedule                  71221
purpose_of_flight         4613 
air_carrier               78865
total_fatal_injuries      26214
total_serious_injuries    28762
total_minor_injuries      27568
total_uninjured           13968
weather_condition         2791 
broad_phase_of_flight     6550 
report_status             0    
publication_date          14073
        

In [1691]:
# dropping unneeded columns
cleaning2=cleaning1.drop(['report_status', 'accident_number','event_id', 'publication_date', 'airport_code','schedule',
            'air_carrier','airport_name',''], axis=1)


In [1692]:
# Now that obviously unnecessary columns have been removed, I'll go through each column one by one to see what is in
# it, removing white spaces and lower casing as I go as an indicator of progress.
cleaning2.columns

Index(['investigation_type', 'event_date', 'location', 'country', 'latitude', 'longitude', 'injury_severity', 'aircraft_damage', 'aircraft_category', 'registration_number', 'make', 'model', 'amateur_built', 'number_of_engines', 'engine_type', 'far_description', 'purpose_of_flight', 'total_fatal_injuries', 'total_serious_injuries', 'total_minor_injuries', 'total_uninjured', 'weather_condition', 'broad_phase_of_flight'], dtype='object')

##   

## Aircraft Category

In [1693]:
cleaning2.aircraft_category.value_counts()

 Airplane              22488
 Helicopter            2781 
 Glider                432  
 Balloon               193  
 Weight-Shift          96   
 Gyrocraft             89   
 Powered Parachute     59   
 Gyroplane             49   
 Ultralight            30   
 Unknown               13   
 Powered-Lift          5    
 Blimp                 4    
 Rocket                1    
Name: aircraft_category, dtype: int64

In [1694]:
# within the aircraft category column, stripping the white space and making them lower case
cleaning2.aircraft_category=[str(i).strip().lower() for i in cleaning2.aircraft_category]

In [1695]:
# checking I have enough information when using helicopters only before dropping other aircraft
(cleaning2.aircraft_category == 'helicopter').value_counts()

False    80196
True     2781 
Name: aircraft_category, dtype: int64

In [1696]:
# subsetting the dataframe to helicopters only
cleaning3=cleaning2[(cleaning2.aircraft_category == 'helicopter')]

In [1697]:
# confirming only helicopter data remains
cleaning3.aircraft_category.value_counts()

helicopter    2781
Name: aircraft_category, dtype: int64

In [1698]:
# can now drop aircraft type as a column as it only contains helicopters
cleaning4=cleaning3.drop('aircraft_category', axis=1)

In [1699]:
cleaning4.shape

(2781, 22)

In [1700]:
cleaning4.head()

,investigation_type,event_date,location,country,latitude,longitude,injury_severity,aircraft_damage,registration_number,make,model,amateur_built,number_of_engines,engine_type,far_description,purpose_of_flight,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight
0,Accident,04/16/2019,"Hauula, HI",United States,21.588333,-157.911944,Non-Fatal,Substantial,N593C,MD HELICOPTER,369,No,1,Turbo Shaft,Part 135: Air Taxi & Commuter,NaN,NaN,NaN,NaN,4,VMC,CRUISE
7,Accident,04/04/2019,"Tampa, FL",United States,27.943333,-82.401944,Fatal(1),Substantial,N4046J,ROBINSON,R44 II,No,1,Reciprocating,Part 91: General Aviation,Personal,1,1,NaN,2,VMC,CRUISE
9,Accident,04/02/2019,"Taunton, MA",United States,41.874445,-71.016389,Non-Fatal,Substantial,N344R,Robinson,R44,No,1,NaN,Part 91: General Aviation,NaN,NaN,NaN,NaN,2,VMC,NaN
20,Accident,03/30/2019,"Montgomery, NY",United States,41.539444,-74.306944,Non-Fatal,Substantial,N222ML,Bell,222,No,2,Turbo Shaft,Part 91: General Aviation,Personal,NaN,NaN,1,5,VMC,TAKEOFF
23,Accident,03/29/2019,"Valley Center, CA",United States,NaN,NaN,Non-Fatal,Substantial,N83181,Bell,206,No,NaN,NaN,Part 137: Agricultural,Aerial Application,NaN,NaN,1,NaN,NaN,NaN


In [1701]:
cleaning4.isna().sum()

investigation_type        1   
event_date                0   
location                  7   
country                   13  
latitude                  612 
longitude                 613 
injury_severity           0   
aircraft_damage           62  
registration_number       445 
make                      3   
model                     2   
amateur_built             42  
number_of_engines         338 
engine_type               329 
far_description           98  
purpose_of_flight         449 
total_fatal_injuries      1872
total_serious_injuries    2053
total_minor_injuries      1922
total_uninjured           1095
weather_condition         348 
broad_phase_of_flight     648 
dtype: int64

##  

## Injuries

In [1702]:
# taking a closer look at injury_severity (0 NaNs) vs other the other injuries columns that have high Nan counts 

In [1703]:
cleaning4.injury_severity=[str(i).strip().lower() for i in cleaning4.injury_severity]


In [1704]:
injury=cleaning4.groupby('injury_severity')['total_fatal_injuries'].value_counts()
pd.DataFrame(injury)

,,total_fatal_injuries
injury_severity,total_fatal_injuries,
fatal(1),1,287
fatal(12),12,1
fatal(13),13,2
fatal(14),14,3
fatal(17),17,1
fatal(2),2,158
fatal(3),3,94
fatal(4),4,49
fatal(5),5,21


In [1705]:
# When injury severity is "non-fatal", total_fatal_injuries is 0.
# It is reasonable to fill total_fatal_injuries NaN's with 0
cleaning4['total_fatal_injuries']= cleaning4.total_fatal_injuries.fillna('0')

In [1706]:
# Assumption- as total_fatal_injuries was populated with NaN when it was 0, then total_serious_injuries, 
# total_minor_injuries and total_uninjured have been populated the same way and can therefore populating 
# NaN values to '0' there as well.



In [1707]:
cleaning4['total_serious_injuries'] = cleaning4['total_serious_injuries'].replace('NaN', np.nan).fillna(0)

In [1708]:
cleaning4['total_minor_injuries'] = cleaning4['total_minor_injuries'].replace('NaN', np.nan).fillna(0)

In [1709]:
cleaning4['total_uninjured'] = cleaning4['total_uninjured'].replace('NaN', np.nan).fillna(0)

In [1710]:
# checking there are now in '0's in the injuries columns.
cleaning4.head()

,investigation_type,event_date,location,country,latitude,longitude,injury_severity,aircraft_damage,registration_number,make,model,amateur_built,number_of_engines,engine_type,far_description,purpose_of_flight,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight
0,Accident,04/16/2019,"Hauula, HI",United States,21.588333,-157.911944,non-fatal,Substantial,N593C,MD HELICOPTER,369,No,1,Turbo Shaft,Part 135: Air Taxi & Commuter,NaN,0,0,0,4,VMC,CRUISE
7,Accident,04/04/2019,"Tampa, FL",United States,27.943333,-82.401944,fatal(1),Substantial,N4046J,ROBINSON,R44 II,No,1,Reciprocating,Part 91: General Aviation,Personal,1,1,0,2,VMC,CRUISE
9,Accident,04/02/2019,"Taunton, MA",United States,41.874445,-71.016389,non-fatal,Substantial,N344R,Robinson,R44,No,1,NaN,Part 91: General Aviation,NaN,0,0,0,2,VMC,NaN
20,Accident,03/30/2019,"Montgomery, NY",United States,41.539444,-74.306944,non-fatal,Substantial,N222ML,Bell,222,No,2,Turbo Shaft,Part 91: General Aviation,Personal,0,0,1,5,VMC,TAKEOFF
23,Accident,03/29/2019,"Valley Center, CA",United States,NaN,NaN,non-fatal,Substantial,N83181,Bell,206,No,NaN,NaN,Part 137: Agricultural,Aerial Application,0,0,1,0,NaN,NaN


In [1711]:
cleaning5=cleaning4.copy()

In [1712]:
# Assumption is that summing across the total fatal, total serious, total minor and total uninjured will give a 
# count of the number of people on board the helicopter at the time of the accident,
# however currently summing is not possible as injury rows are strings.
type(cleaning5.total_fatal_injuries[0])

str

In [1713]:
# Changing strings to integers ready for summing.

cleaning5.total_fatal_injuries=[int(i) for i in cleaning5.total_fatal_injuries]
cleaning5.total_serious_injuries=[int(i) for i in cleaning5.total_serious_injuries]
cleaning5.total_minor_injuries=[int(i) for i in cleaning5.total_minor_injuries]
cleaning5.total_uninjured=[int(i) for i in cleaning5.total_uninjured]

In [1714]:
# Checking values have changed to integers.
type(cleaning5.total_fatal_injuries[0])

numpy.int64

In [1715]:
cleaning5.injury_severity.value_counts()

non-fatal      2040
fatal(1)       287 
fatal(2)       158 
fatal(3)       94  
incident       79  
fatal(4)       49  
unavailable    25  
fatal(5)       21  
fatal(9)       7   
fatal(7)       6   
fatal(6)       5   
fatal(14)      3   
fatal(8)       3   
fatal(13)      2   
fatal(12)      1   
fatal(17)      1   
Name: injury_severity, dtype: int64

In [1716]:
# Checking what is in the dataset when the injury type is unavailable.
#  There are no values in the injuries column to work with so dropping rows where injury_severity is unavailable.
cleaning5[cleaning5['injury_severity'] == 'unavailable']

,investigation_type,event_date,location,country,latitude,longitude,injury_severity,aircraft_damage,registration_number,make,model,amateur_built,number_of_engines,engine_type,far_description,purpose_of_flight,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight
138,Accident,02/18/2019,"Glennallen, AK",United States,NaN,NaN,unavailable,NaN,N350ST,AIRBUS,A350,No,1,NaN,Part 135: Air Taxi & Commuter,NaN,0,0,0,0,NaN,NaN
247,Accident,01/02/2019,"Santa Maria Island, FL",United States,27.483334,-82.733334,unavailable,Destroyed,N772MG,Robinson,R44,No,1,NaN,Part 91: General Aviation,NaN,0,0,0,0,NaN,NaN
265,Accident,12/24/2018,"Strum, WI",United States,NaN,NaN,unavailable,NaN,N145SM,Eurocopter Deutschland,BK117,No,NaN,NaN,Part 135: Air Taxi & Commuter,NaN,0,0,0,0,NaN,NaN
707,Accident,08/29/2018,"Girdwood, AK",United States,60.284444,-152.453333,unavailable,Substantial,N99645,Robinson,R44,No,1,NaN,Part 91: General Aviation,Business,0,0,0,0,NaN,NaN
772,Accident,08/18/2018,"Battle Mountain, NV",United States,NaN,NaN,unavailable,NaN,N8052G,Bell,206,No,NaN,NaN,Part 91: General Aviation,Aerial Observation,0,0,0,0,NaN,NaN
1730,Accident,02/01/2018,"Marchwood, New Hampshire, United Kingdom",United Kingdom,NaN,NaN,unavailable,Substantial,NaN,SIKORSKY,S61,No,NaN,NaN,Unknown,NaN,0,0,0,0,NaN,TAKEOFF
2751,Accident,06/22/2017,"Alpbach, Austria",Austria,47.399166,101.943333,unavailable,Substantial,NaN,BELL,204,No,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN
3478,Accident,12/12/2016,"Cannes, France",France,NaN,NaN,unavailable,Destroyed,NaN,ROBINSON,R22,No,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN
4011,Accident,08/16/2016,"Czech Republic, Czech Republic",Czech Republic,0.000000,0.000000,unavailable,Substantial,NaN,ROBINSON,R44,No,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN
4595,Accident,05/08/2016,"Mexico, Mexico",Mexico,NaN,NaN,unavailable,NaN,N531RR,BELL,206,No,NaN,NaN,"Non-U.S., Non-Commercial",NaN,0,0,0,0,NaN,NaN


In [1717]:
cleaning5.shape

(2781, 22)

In [1718]:
#  removing rows where injury severity is unavailable.
cleaning6=cleaning5[(cleaning5.injury_severity != 'unavailable')]

In [1719]:
#  checking that 25 rows have been removed
cleaning6.shape

(2756, 22)

In [1720]:
# Taking a look at the dataframe when the injury severity = incident.
# I note that when incident is noted in the injury severity column, it is also investigation_type incident
# therefore will address this later when looking at incident vs accident.
cleaning6[cleaning6.injury_severity == 'incident']

,investigation_type,event_date,location,country,latitude,longitude,injury_severity,aircraft_damage,registration_number,make,model,amateur_built,number_of_engines,engine_type,far_description,purpose_of_flight,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight
1741,Incident,01/29/2018,"Shetland, United Kingdom",United Kingdom,NaN,NaN,incident,NaN,NaN,SIKORSKY,S-92A,No,NaN,NaN,"Non-U.S., Commercial",NaN,0,0,0,21,NaN,LANDING
1744,Incident,01/28/2018,"Memphis, TN",United States,35.136944,-90.025000,incident,Minor,N894GT,AIRBUS HELICOPTERS,EC 130,No,1,Turbo Shaft,Part 135: Air Taxi & Commuter,NaN,0,0,0,4,VMC,NaN
2198,Incident,09/21/2017,"Hoffman Island, NY",United States,40.500000,-74.000000,incident,Minor,R20087,SIKORSKY,UH60,No,NaN,NaN,Armed Forces,NaN,0,0,0,5,VMC,CRUISE
2199,Incident,09/21/2017,"Hoffman Island, NY",United States,40.500000,-74.000000,incident,Destroyed,None,DJI,Phantom,No,4,Electric,Part 107: Small UAS,NaN,0,0,0,5,VMC,CRUISE
2351,Incident,08/22/2017,"Glendale, AZ",United States,33.526945,-112.295277,incident,NaN,N52PD,ENSTROM,F 28F,No,1,Reciprocating,Part 91: General Aviation,Instructional,0,0,0,2,VMC,NaN
3317,Incident,02/07/2017,"Locarno, Switzerland",Switzerland,NaN,NaN,incident,NaN,NaN,SCHWEIZER,269C,No,NaN,NaN,"Non-U.S., Non-Commercial",NaN,0,0,0,2,NaN,MANEUVERING
3356,Incident,01/26/2017,"Sioux Falls, SD",United States,43.578334,-96.748055,incident,Minor,N911MK,EUROCOPTER DEUTSCHLAND GMBH,MBB BK 117 C-2,No,2,Turbo Shaft,Part 135: Air Taxi & Commuter,NaN,0,0,0,3,VMC,CRUISE
3437,Incident,12/28/2016,"North Sea, UK, United Kingdom",United Kingdom,56.992777,2.136111,incident,NaN,NaN,SIKORSKY,S92,No,NaN,Turbo Shaft,NaN,NaN,0,0,0,11,NaN,LANDING
3734,Incident,10/06/2016,"Wanaka, New Zealand",New Zealand,NaN,NaN,incident,Minor,NaN,MD HELICOPTER,500,No,NaN,Turbo Shaft,NaN,NaN,0,0,0,0,NaN,STANDING
3778,Incident,09/28/2016,"Ballina, Australia",Australia,NaN,NaN,incident,NaN,NaN,PULSE,Vapor 55,No,NaN,NaN,NaN,NaN,0,0,0,0,NaN,CRUISE


##  

## People on Board

In [1721]:
# Assumption- summing the injuryt columns will give the total number of people in the helicopter.
# Making a new column of how many people on board.
cleaning5['people_on_board']=cleaning5.total_fatal_injuries + cleaning5.total_serious_injuries + cleaning5.total_minor_injuries +cleaning5.total_uninjured
cleaning5

,investigation_type,event_date,location,country,latitude,longitude,injury_severity,aircraft_damage,registration_number,make,model,amateur_built,number_of_engines,engine_type,far_description,purpose_of_flight,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight,people_on_board
0,Accident,04/16/2019,"Hauula, HI",United States,21.588333,-157.911944,non-fatal,Substantial,N593C,MD HELICOPTER,369,No,1,Turbo Shaft,Part 135: Air Taxi & Commuter,NaN,0,0,0,4,VMC,CRUISE,4
7,Accident,04/04/2019,"Tampa, FL",United States,27.943333,-82.401944,fatal(1),Substantial,N4046J,ROBINSON,R44 II,No,1,Reciprocating,Part 91: General Aviation,Personal,1,1,0,2,VMC,CRUISE,4
9,Accident,04/02/2019,"Taunton, MA",United States,41.874445,-71.016389,non-fatal,Substantial,N344R,Robinson,R44,No,1,NaN,Part 91: General Aviation,NaN,0,0,0,2,VMC,NaN,2
20,Accident,03/30/2019,"Montgomery, NY",United States,41.539444,-74.306944,non-fatal,Substantial,N222ML,Bell,222,No,2,Turbo Shaft,Part 91: General Aviation,Personal,0,0,1,5,VMC,TAKEOFF,6
23,Accident,03/29/2019,"Valley Center, CA",United States,NaN,NaN,non-fatal,Substantial,N83181,Bell,206,No,NaN,NaN,Part 137: Agricultural,Aerial Application,0,0,1,0,NaN,NaN,1
40,Accident,03/22/2019,"Blazkov village, Czech Republic., Czech Republic",Czech Republic,NaN,NaN,fatal(2),Substantial,NaN,ENSTROM,480,No,NaN,NaN,"Non-U.S., Non-Commercial",Instructional,2,0,0,0,NaN,MANEUVERING,2
47,Accident,03/20/2019,"Madill, OK",United States,NaN,NaN,non-fatal,Substantial,N7782H,Robinson,R22,No,1,Reciprocating,Part 91: General Aviation,Personal,0,0,0,1,VMC,CRUISE,1
51,Accident,03/17/2019,"Newberg, OR",United States,NaN,NaN,non-fatal,Substantial,N119T,MD HELICOPTER,369,No,1,Turbo Shaft,Part 91: General Aviation,Personal,0,0,0,2,VMC,CRUISE,2
62,Accident,03/13/2019,"Keene, NH",United States,42.898333,-72.270834,non-fatal,Substantial,N8045H,Hughes,TH 55,No,1,Reciprocating,Part 91: General Aviation,Personal,0,0,0,1,VMC,CRUISE,1
65,Accident,03/13/2019,"Ponte Vedra Beach, FL",United States,30.206111,-81.403333,non-fatal,Substantial,N519P,Bell,OH 58A,No,NaN,Turbo Shaft,Part 91: General Aviation,Public Aircraft - Local,0,0,0,2,VMC,MANEUVERING,2


In [1722]:
cleaning5.people_on_board.value_counts()

1      995
2      994
3      356
4      199
5      81 
0      37 
6      37 
7      34 
9      12 
8      7  
13     5  
11     5  
14     5  
10     4  
18     3  
20     2  
12     2  
21     1  
140    1  
133    1  
Name: people_on_board, dtype: int64

In [1723]:
# Clearly the original people on board assumption is incorrect with having 133 POB in a helicopter! 
# Check out what this is- Note this in an 'incident'.
cleaning5.loc[cleaning5['people_on_board'] == 133]

,investigation_type,event_date,location,country,latitude,longitude,injury_severity,aircraft_damage,registration_number,make,model,amateur_built,number_of_engines,engine_type,far_description,purpose_of_flight,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight,people_on_board
14015,Incident,09/04/2010,"Los Angeles, CA",United States,NaN,NaN,incident,NaN,NaN,BELL,222U,No,2,Turbo Shaft,Part 91: General Aviation,Positioning,0,0,0,133,VMC,NaN,133


##  

## Accident vs Incident

In [1724]:
# https://en.wikipedia.org/wiki/Bell_222/230 states that a Bell 222 can hold a maximum of 10 people.
# Note that this is an "incident" and no one has been hurt. It seems that anyone in the vacinity has been included 
# in the count.
# As the project question is about accidents, decision to drop incident rows.
cleaning5.investigation_type.value_counts()

 Accident     2701
 Incident     79  
Name: investigation_type, dtype: int64

In [1725]:
cleaning5.investigation_type=[str(i).strip().lower() for i in cleaning5.investigation_type]

In [1726]:
# Dropping incidents so the dataframe contains accidents only
cleaning5=cleaning5[cleaning5.investigation_type != "incident"]

In [1727]:
# And another check where accidents had more than eight people on board.
cleaning5.loc[cleaning5['people_on_board'] >8 ]
# Bell 412 -up to 13 passengers
# Sirkorsky s76 - two pilots, 12 passengers
# AW139-two pilots, up to 15 passengers
# Bell 214-two pilots, up to 14 passengers

,investigation_type,event_date,location,country,latitude,longitude,injury_severity,aircraft_damage,registration_number,make,model,amateur_built,number_of_engines,engine_type,far_description,purpose_of_flight,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight,people_on_board
822,accident,08/10/2018,"Nakanojo, Japan",Japan,36.646389,138.845555,fatal(9),Destroyed,NaN,BELL,412EP,No,NaN,NaN,Unknown,NaN,9,0,0,0,NaN,MANEUVERING,9
1828,accident,12/30/2017,"Alicante, Spain",Spain,NaN,NaN,fatal(1),Substantial,NaN,BELL,412SP,No,NaN,NaN,"Non-U.S., Commercial",NaN,1,0,0,10,NaN,APPROACH,11
3208,accident,03/05/2017,"Matsumoto, Japan",Japan,36.233889,138.048889,fatal(9),Substantial,NaN,BELL,412,No,NaN,NaN,NaN,NaN,9,0,0,0,NaN,NaN,9
4641,accident,04/28/2016,"Campos dos Goytacazes, Brazil",Brazil,-21.790833,-41.306111,non-fatal,Substantial,NaN,SIKORSKY,S76,No,NaN,NaN,NaN,NaN,0,0,0,11,NaN,NaN,11
4642,accident,04/28/2016,"Campos dos Goytacazes, Brazil",Brazil,-21.790833,-41.306111,non-fatal,Substantial,NaN,SIKORSKY,S76,No,2,Turbo Shaft,"Non-U.S., Commercial",NaN,0,0,0,11,NaN,DESCENT,11
4731,accident,04/07/2016,"Galliano, LA",United States,29.438055,-90.262500,non-fatal,Substantial,N639NA,AGUSTAWESTLAND PHILADELPHIA,AW139,No,2,Turbo Shaft,Part 91: General Aviation,Other Work Use,0,0,0,9,VMC,CRUISE,9
5675,accident,08/08/2015,"Istra, Russia",Russia,NaN,NaN,fatal(9),Destroyed,NaN,ROBINSON,R44,No,1,Reciprocating,"Non-U.S., Non-Commercial",Unknown,9,0,0,0,NaN,MANEUVERING,9
7323,accident,07/25/2014,"Boulder City, NV",United States,35.947500,-114.855000,non-fatal,Substantial,N154GC,EUROCOPTER,EC 130 B4,No,1,Turbo Shaft,Part 135: Air Taxi & Commuter,NaN,0,0,0,9,VMC,LANDING,9
8256,accident,12/05/2013,"Port Fourchon, LA",United States,29.929723,-90.353889,non-fatal,Substantial,N707MY,SIKORSKY,S 76B,No,2,Turbo Shaft,Part 135: Air Taxi & Commuter,NaN,0,0,0,10,VMC,TAKEOFF,10
8274,accident,11/29/2013,"Glasgow, United Kingdom",United Kingdom,NaN,NaN,fatal(9),Substantial,NaN,EUROCOPTER,EC135,No,NaN,NaN,"Non-U.S., Non-Commercial",NaN,9,0,0,0,NaN,NaN,9


In [1728]:
# And a 2nd check.
cleaning5.loc[cleaning5['people_on_board'] == 18]
# Sikorsky S92a can carry 18 passengers
# EC225 can carry 24 passengers
# I'm now satisfied with the people on board column

,investigation_type,event_date,location,country,latitude,longitude,injury_severity,aircraft_damage,registration_number,make,model,amateur_built,number_of_engines,engine_type,far_description,purpose_of_flight,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight,people_on_board
16877,accident,03/12/2009,"St. John's, Newfoundland, Canada",Canada,NaN,NaN,fatal(17),Destroyed,NaN,SIKORSKY,S-92A,NaN,NaN,NaN,"Non-U.S., Commercial",NaN,17,1,0,0,NaN,NaN,18
16979,accident,02/18/2009,"North Sea, United Kingdom",United Kingdom,NaN,NaN,non-fatal,Substantial,NaN,EUROCOPTER,EC225,No,NaN,NaN,"Non-U.S., Non-Commercial",Unknown,0,0,0,18,VMC,NaN,18


In [1729]:
# Checking only accidents remain
cleaning5.investigation_type.value_counts()

accident    2701
nan         1   
Name: investigation_type, dtype: int64

In [1730]:
# Taking a closer look at the nan that is still appearing
cleaning5['investigation_type']=cleaning5['investigation_type'].replace('nan', 'find')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [1731]:
cleaning5.loc[cleaning5['investigation_type'] == 'find']

,investigation_type,event_date,location,country,latitude,longitude,injury_severity,aircraft_damage,registration_number,make,model,amateur_built,number_of_engines,engine_type,far_description,purpose_of_flight,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight,people_on_board
745,find,08/22/2018,"Ridgeway, PA",United States,41.401111,-78.561389,non-fatal,Substantial,N89ZC,Hughes,369,No,NaN,NaN,Part 91: General Aviation,Aerial Observation,0,0,0,2,NaN,NaN,2


In [1732]:
# As the aircraft_damage is substantial, I will count this as an accident.
cleaning5['investigation_type']= cleaning5['investigation_type'].replace('find', 'accident')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [1733]:
cleaning5.investigation_type.value_counts()

accident    2702
Name: investigation_type, dtype: int64

##  

In [1734]:
cleaning5.columns

Index(['investigation_type', 'event_date', 'location', 'country', 'latitude', 'longitude', 'injury_severity', 'aircraft_damage', 'registration_number', 'make', 'model', 'amateur_built', 'number_of_engines', 'engine_type', 'far_description', 'purpose_of_flight', 'total_fatal_injuries', 'total_serious_injuries', 'total_minor_injuries', 'total_uninjured', 'weather_condition', 'broad_phase_of_flight', 'people_on_board'], dtype='object')

In [1735]:
# changing the order of the columns for better visibility of columns I'm now working on.
cleaning6=cleaning5[['investigation_type', 'event_date', 'location', 'country', 'latitude',
       'longitude', 'make', 'model', 'broad_phase_of_flight',
       'amateur_built', 'number_of_engines', 'engine_type', 'far_description',
       'purpose_of_flight', 'injury_severity', 'aircraft_damage', 'weather_condition','total_fatal_injuries', 'total_serious_injuries',
       'total_minor_injuries', 'total_uninjured', 'people_on_board', 
        'registration_number']]

In [1736]:
cleaning6.isnull().sum()

investigation_type        0  
event_date                0  
location                  6  
country                   13 
latitude                  584
longitude                 585
make                      3  
model                     2  
broad_phase_of_flight     628
amateur_built             39 
number_of_engines         320
engine_type               316
far_description           84 
purpose_of_flight         405
injury_severity           0  
aircraft_damage           34 
weather_condition         326
total_fatal_injuries      0  
total_serious_injuries    0  
total_minor_injuries      0  
total_uninjured           0  
people_on_board           0  
registration_number       418
dtype: int64

In [1737]:
# Checking to see shape if all NaNs were dropped at this point- will not dropna at this stage. 
# Will keep trying to resolve NaNs
cleaning6.dropna().shape

(1364, 23)

##  

## Make and Model

In [1738]:
# cleaning make and model columns
cleaning6.make=[str(i).strip().lower() for i in cleaning6.make]

In [1739]:
cleaning6.model=[str(i).strip().lower() for i in cleaning6.model]

In [1740]:
type(cleaning6.make[0])

str

In [1741]:
# Notice that the make eurocopter has several different variations
cleaning6[cleaning6['make'].str.contains('euro')]

,investigation_type,event_date,location,country,latitude,longitude,make,model,broad_phase_of_flight,amateur_built,number_of_engines,engine_type,far_description,purpose_of_flight,injury_severity,aircraft_damage,weather_condition,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,people_on_board,registration_number
177,accident,01/29/2019,"Tampa, FL",United States,28.011111,-82.344444,eurocopter,as 350,LANDING,No,1,Turbo Shaft,Public Aircraft,Instructional,non-fatal,Substantial,VMC,0,0,0,2,2,N680DG
265,accident,12/24/2018,"Strum, WI",United States,NaN,NaN,eurocopter deutschland,bk117,NaN,No,NaN,NaN,Part 135: Air Taxi & Commuter,NaN,unavailable,NaN,NaN,0,0,0,0,0,N145SM
349,accident,11/22/2018,"La Romana, Dominican Republic",Dominican Republic,18.497222,-68.958333,eurocopter,ec120,APPROACH,No,1,Turbo Shaft,"Non-U.S., Commercial",NaN,fatal(5),Destroyed,UNK,5,0,0,0,5,N124ML
557,accident,09/29/2018,"Ruidoso, NM",United States,33.396666,-105.785833,eurocopter,as350,NaN,No,1,NaN,Part 135: Air Taxi & Commuter,NaN,non-fatal,Substantial,VMC,0,0,2,0,2,N894NA
764,accident,08/19/2018,"Hilo, HI",United States,19.643611,-154.984444,eurocopter,ec130,NaN,No,1,Turbo Shaft,Part 91: General Aviation,Other Work Use,non-fatal,Substantial,NaN,0,0,1,1,2,N11QP
925,accident,07/25/2018,"Kobuk, AK",United States,66.935555,-156.773056,eurocopter,as 350 b3,NaN,No,NaN,NaN,Part 135: Air Taxi & Commuter,NaN,non-fatal,Substantial,NaN,0,0,1,0,1,N354LA
1033,accident,07/07/2018,"Chicago, IL",United States,41.715278,-87.624445,eurocopter deutschland gmbh,ec135p1,MANEUVERING,No,2,Turbo Shaft,Part 135: Air Taxi & Commuter,NaN,non-fatal,Substantial,VMC,0,1,2,1,4,N312SA
1443,accident,04/26/2018,"Hazelhurst, WI",United States,45.754444,-89.695833,eurocopter,as 350 b2,CRUISE,No,1,Turbo Shaft,Part 91: General Aviation,Positioning,fatal(3),Destroyed,VMC,3,0,0,0,3,N127LN
1567,accident,03/21/2018,"Hardy Reef, Queensland, Australia",Australia,NaN,NaN,eurocopter,ec120,GO-AROUND,No,1,Turbo Shaft,Unknown,NaN,fatal(2),Substantial,NaN,2,0,2,1,5,NaN
1602,accident,03/11/2018,"New York, NY",United States,40.773611,-73.939444,american eurocopter corp,as350b2,STANDING,No,1,Turbo Shaft,Part 91: General Aviation,Aerial Observation,fatal(5),Substantial,VMC,5,0,1,0,6,N350LH


In [1742]:
# Resetting index before heading into substantial cleaning.
cleaning6.reset_index(drop=True, inplace=True)

In [1743]:
cleaning6['make'].replace({'eurocopter deutschland': 'eurocopter'}, inplace = True)
cleaning6['make'].replace({'eurocopter deutschland gmbh': 'eurocopter'}, inplace = True)
cleaning6['make'].replace({'eurocopter france': 'eurocopter'}, inplace = True)
cleaning6['make'].replace({'american eurocopter': 'eurocopter'}, inplace = True)
cleaning6['make'].replace({'american eurocopter corp': 'eurocopter'}, inplace = True)


In [1744]:
# checking it has worked
cleaning6[cleaning6['make'].str.contains('euro')]

,investigation_type,event_date,location,country,latitude,longitude,make,model,broad_phase_of_flight,amateur_built,number_of_engines,engine_type,far_description,purpose_of_flight,injury_severity,aircraft_damage,weather_condition,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,people_on_board,registration_number
21,accident,01/29/2019,"Tampa, FL",United States,28.011111,-82.344444,eurocopter,as 350,LANDING,No,1,Turbo Shaft,Public Aircraft,Instructional,non-fatal,Substantial,VMC,0,0,0,2,2,N680DG
33,accident,12/24/2018,"Strum, WI",United States,NaN,NaN,eurocopter,bk117,NaN,No,NaN,NaN,Part 135: Air Taxi & Commuter,NaN,unavailable,NaN,NaN,0,0,0,0,0,N145SM
38,accident,11/22/2018,"La Romana, Dominican Republic",Dominican Republic,18.497222,-68.958333,eurocopter,ec120,APPROACH,No,1,Turbo Shaft,"Non-U.S., Commercial",NaN,fatal(5),Destroyed,UNK,5,0,0,0,5,N124ML
62,accident,09/29/2018,"Ruidoso, NM",United States,33.396666,-105.785833,eurocopter,as350,NaN,No,1,NaN,Part 135: Air Taxi & Commuter,NaN,non-fatal,Substantial,VMC,0,0,2,0,2,N894NA
83,accident,08/19/2018,"Hilo, HI",United States,19.643611,-154.984444,eurocopter,ec130,NaN,No,1,Turbo Shaft,Part 91: General Aviation,Other Work Use,non-fatal,Substantial,NaN,0,0,1,1,2,N11QP
101,accident,07/25/2018,"Kobuk, AK",United States,66.935555,-156.773056,eurocopter,as 350 b3,NaN,No,NaN,NaN,Part 135: Air Taxi & Commuter,NaN,non-fatal,Substantial,NaN,0,0,1,0,1,N354LA
114,accident,07/07/2018,"Chicago, IL",United States,41.715278,-87.624445,eurocopter,ec135p1,MANEUVERING,No,2,Turbo Shaft,Part 135: Air Taxi & Commuter,NaN,non-fatal,Substantial,VMC,0,1,2,1,4,N312SA
154,accident,04/26/2018,"Hazelhurst, WI",United States,45.754444,-89.695833,eurocopter,as 350 b2,CRUISE,No,1,Turbo Shaft,Part 91: General Aviation,Positioning,fatal(3),Destroyed,VMC,3,0,0,0,3,N127LN
169,accident,03/21/2018,"Hardy Reef, Queensland, Australia",Australia,NaN,NaN,eurocopter,ec120,GO-AROUND,No,1,Turbo Shaft,Unknown,NaN,fatal(2),Substantial,NaN,2,0,2,1,5,NaN
172,accident,03/11/2018,"New York, NY",United States,40.773611,-73.939444,eurocopter,as350b2,STANDING,No,1,Turbo Shaft,Part 91: General Aviation,Aerial Observation,fatal(5),Substantial,VMC,5,0,1,0,6,N350LH


In [1745]:
# Looking at the variations of make names in order to replace to a standardised naming convention.
cleaning6.make.value_counts()

bell                              717
robinson                          386
hughes                            215
robinson helicopter               199
eurocopter                        163
robinson helicopter company       142
schweizer                         101
aerospatiale                      77 
enstrom                           66 
sikorsky                          62 
hiller                            54 
mcdonnell douglas helicopter      27 
md helicopter                     25 
agusta                            20 
bell helicopter textron           18 
brantly                           17 
airbus                            17 
rotorway                          17 
mcdonnell douglas                 14 
bell helicopter textron canada    11 
robinson helicopter co            11 
texas helicopter corp             10 
mcdonnell douglas heli co         10 
mbb                               9  
garlick                           8  
md helicopter inc                 7  
airbus helic

In [1746]:
# putting an underscore into the names to make them easier to deal with
cleaning6.make=[i.replace(' ','_') for i in cleaning6.make]

In [1747]:
cleaning6['make'].replace({'robinson_helicopter': 'robinson'}, inplace = True)
cleaning6['make'].replace({'robinson_helicopter_company': 'robinson'}, inplace = True)
cleaning6['make'].replace({'mcdonnell_douglas_helicopter': 'md_helicopter'}, inplace = True)
cleaning6['make'].replace({'bell_helicopter_textron ': 'bell'}, inplace = True)
cleaning6['make'].replace({'mcdonnell_douglas': 'md_helicopter'}, inplace = True)
cleaning6['make'].replace({'bell_helicopter_textron_canada': 'bell'}, inplace = True)
cleaning6['make'].replace({'robinson_helicopter_co ': 'robinson'}, inplace = True)
cleaning6['make'].replace({'mcdonnell_douglas_heli_co': 'md_helicopter'}, inplace = True)
cleaning6['make'].replace({'texas_helicopter_corp': 'bell'}, inplace = True)
cleaning6['make'].replace({'md_helicopter_inc': 'md_helicopter'}, inplace = True)
cleaning6['make'].replace({'mbb': 'bolkow'}, inplace = True)
cleaning6['make'].replace({'airbus_helicopters': 'eurocopter'}, inplace = True)
cleaning6['make'].replace({'continental_copters_inc.': 'eurocopter'}, inplace = True)
cleaning6['make'].replace({'bell_helicopter': 'bell'}, inplace = True)
cleaning6['make'].replace({'texas_helicopter_corporation': 'bell'}, inplace = True)
cleaning6['make'].replace({'mcdonnell_douglas_helicopter_c': 'md_helicopter'}, inplace = True)
cleaning6['make'].replace({'bell/soloy': 'bell'}, inplace = True)
cleaning6['make'].replace({'bell-carson': 'bell'}, inplace = True)
cleaning6['make'].replace({'messerschmitt-boelkow-blohm': 'bolkow'}, inplace = True)
cleaning6['make'].replace({'bell_helicopter_textron': 'bell'}, inplace = True)
cleaning6['make'].replace({'robinson_helicopter_co': 'robinson'}, inplace = True)
cleaning6['make'].replace({'enstrom_helicopter_corp': 'enstrom'}, inplace = True)
cleaning6['make'].replace({'garlick_helicopters_inc': 'garlick'}, inplace = True)
cleaning6['make'].replace({'messerschmitt-bolkow-blohm': 'bolkow'}, inplace = True)
cleaning6['make'].replace({'american_eurocopter_llc': 'eurocopter'}, inplace = True)
cleaning6['make'].replace({'fairchild_hiller': 'hiller'}, inplace = True)
cleaning6['make'].replace({'brantly_helicopter': 'brantly'}, inplace = True)
cleaning6['make'].replace({'agustawestland_philadelphia': 'agusta'}, inplace = True)
cleaning6['make'].replace({'agusta_spa': 'agusta'}, inplace = True)
cleaning6['make'].replace({'md_helicopters_inc': 'md_helicopter'}, inplace = True)
cleaning6['make'].replace({'helicopteres_guimbal': 'guimbal'}, inplace = True)
cleaning6['make'].replace({'airbus_helicopters_inc': 'eurocopter'}, inplace = True)
cleaning6['make'].replace({'bell_helicopter_co ': 'bell'}, inplace = True)
cleaning6['make'].replace({'robinson_helicopter_co_inc': 'robinson'}, inplace = True)
cleaning6['make'].replace({'hughes_helicopters_inc': 'hughes'}, inplace = True)
cleaning6['make'].replace({'agustawestland_spa': 'agusta'}, inplace = True)
cleaning6['make'].replace({'sikorsky_aircraft_corporation': 'sikorsky'}, inplace = True)
cleaning6['make'].replace({'bell-cont_42g': 'bell'}, inplace = True)
cleaning6['make'].replace({'brecher_rotorway ': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'scorpion_too': 'scorpion'}, inplace = True)
cleaning6['make'].replace({'bölkow': 'bolkow'}, inplace = True)
cleaning6['make'].replace({'texas_helicopter': 'bell'}, inplace = True)
cleaning6['make'].replace({'kaman_aerospace_corp': 'kaman'}, inplace = True)
cleaning6['make'].replace({'augusta': 'agusta'}, inplace = True)
cleaning6['make'].replace({'bell_helicopter_co': 'bell'}, inplace = True)
cleaning6['make'].replace({'mcdonnell_douglas_helicopters': 'md_helicopter'}, inplace = True)
cleaning6['make'].replace({'hughes/helicopter_assocs_inc': 'hughes'}, inplace = True)
cleaning6['make'].replace({'brecher_rotorway': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'arrow_falcon_exporters,_inc.': 'bell'}, inplace = True)
cleaning6['make'].replace({'vertical_aviation_technologies': 'hummingbird'}, inplace = True)
cleaning6['make'].replace({'arrow_falcon_exporters_inc': 'bell'}, inplace = True)
cleaning6['make'].replace({'williams_helicopter_corp': 'bell'}, inplace = True)
cleaning6['make'].replace({'hiller-soloy': 'hiller'}, inplace = True)
cleaning6['make'].replace({'md_helicopters': 'md_helicopter'}, inplace = True)
cleaning6['make'].replace({'agusta_aerospace_corp': 'agusta'}, inplace = True)
cleaning6['make'].replace({'boeing_helicopters_div.': 'boeing'}, inplace = True)
cleaning6['make'].replace({'texas_helicopter_corp.': 'bell'}, inplace = True)
cleaning6['make'].replace({'sud_aviation': 'aerospatiale'}, inplace = True)
cleaning6['make'].replace({'moore_bell ': 'bell'}, inplace = True)
cleaning6['make'].replace({'mdhi ': 'md_helicopter'}, inplace = True)
cleaning6['make'].replace({'s.n.i.a.s.': 'aerospatiale'}, inplace = True)
cleaning6['make'].replace({'scorpion': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'smith': 'bell'}, inplace = True)
cleaning6['make'].replace({'bell/textron': 'bell'}, inplace = True)
cleaning6['make'].replace({'airbus': 'eurocopter'}, inplace = True)
cleaning6['make'].replace({'bosko_wr/bosko_ce': 'safari'}, inplace = True)
cleaning6['make'].replace({'kawasaki_heavy_industries_ltd.': 'kawaski'}, inplace = True)
cleaning6['make'].replace({'schweizer_300cb': 'schweizer'}, inplace = True)
cleaning6['make'].replace({'schweizer_300cbi': 'schweizer'}, inplace = True)
cleaning6['make'].replace({'agusta_bell': 'bell'}, inplace = True)
cleaning6['make'].replace({'western_international_avia_inc': 'bell'}, inplace = True)
cleaning6['make'].replace({'leonardo': 'agusta'}, inplace = True)
cleaning6['make'].replace({'gaddis_michael': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'schweizer(hughes)': 'hughes'}, inplace = True)
cleaning6['make'].replace({'bell-tellijohn': 'bell'}, inplace = True)
cleaning6['make'].replace({'rotorcraft_development_corp': 'bell'}, inplace = True)
cleaning6['make'].replace({'bell-transworld_helicopters': 'bell'}, inplace = True)
cleaning6['make'].replace({'means_rober_c': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'gray_jim_robert': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'unknown': 'hummingbird'}, inplace = True)
cleaning6['make'].replace({'helicycle/morrisey_rr': 'helicycle'}, inplace = True)
cleaning6['make'].replace({'bell_helicopter_textron_inc': 'bell'}, inplace = True)
cleaning6['make'].replace({'richard_fuist': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'giackino_donald_w': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'airbus_helicopters_(eurocopte': 'eurocopter'}, inplace = True)
cleaning6['make'].replace({'bell/garlick': 'bell'}, inplace = True)
cleaning6['make'].replace({'butler_raymond': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'bell-watson': 'bell'}, inplace = True)
cleaning6['make'].replace({'judr_zdenek_prazdny': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'eagle_r&d_inc': 'helicycle'}, inplace = True)
cleaning6['make'].replace({'agustawestland': 'agusta'}, inplace = True)
cleaning6['make'].replace({'mdhi': 'hughes'}, inplace = True)
cleaning6['make'].replace({'airbus/eurocopter': 'eurocopter'}, inplace = True)
cleaning6['make'].replace({'emery_mark': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'sands': 'helicycle'}, inplace = True)
cleaning6['make'].replace({'siemens': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'bell-transworld_helicopter_cor': 'bell'}, inplace = True)
cleaning6['make'].replace({'continental_copters_inc': 'eurocopter'}, inplace = True)
cleaning6['make'].replace({'versteeg': 'helicycle'}, inplace = True)
cleaning6['make'].replace({'helicycle': 'eagle'}, inplace = True)
cleaning6['make'].replace({'innovator_technologies': 'mosquito'}, inplace = True)
cleaning6['make'].replace({'erickson_air_crane': 'sikorsky'}, inplace = True)
cleaning6['make'].replace({'hiller-tri-plex_ind.inc.': 'hiller'}, inplace = True)
cleaning6['make'].replace({'goetz': 'eagle'}, inplace = True)
cleaning6['make'].replace({'croman_corporation': 'sikorsky'}, inplace = True)
cleaning6['make'].replace({'scotts-bell_47_inc': 'bell'}, inplace = True)
cleaning6['make'].replace({'omlin_robert_g': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'faltin_john_r': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'rotorcraft_developement_corp.': 'bell'}, inplace = True)
cleaning6['make'].replace({'ruiz_e': 'raf'}, inplace = True)
cleaning6['make'].replace({'pzl-swidnik': 'pzl'}, inplace = True)
cleaning6['make'].replace({'schweizer(hughes)aircraft_corp': 'schweizer'}, inplace = True)
cleaning6['make'].replace({'sud_aviation/eurocopter': 'sud_aviation'}, inplace = True)
cleaning6['make'].replace({'hyde_william_r': 'eagle'}, inplace = True)
cleaning6['make'].replace({'md_helicopters,_inc.': 'md_helicopter'}, inplace = True)
cleaning6['make'].replace({'d_&_d_aircraft': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'burr_andrew_e_xiv': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'scotts-bell_47,_inc.': 'bell'}, inplace = True)
cleaning6['make'].replace({'hiller-rogerson_helicopter': 'hiller'}, inplace = True)
cleaning6['make'].replace({'worthington': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'continental_copters': 'continental'}, inplace = True)
cleaning6['make'].replace({'johnson': 'eagle'}, inplace = True)
cleaning6['make'].replace({'agustadwestland': 'agusta'}, inplace = True)
cleaning6['make'].replace({'erickson': 'sikorsky'}, inplace = True)
cleaning6['make'].replace({'seymour_robert_m': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'smith_richard_d_jr': 'revolution'}, inplace = True)
cleaning6['make'].replace({'piasecki/pike': 'piasecki'}, inplace = True)
cleaning6['make'].replace({'boykin_stephen_vance': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'yates_w/lazar_m': 'safari'}, inplace = True)
cleaning6['make'].replace({'bingham_chuck': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'bielawa': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'croman': 'sikorsky'}, inplace = True)
cleaning6['make'].replace({'bolton_ricky_d': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'york_ronald_j': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'sidler_mike': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'better_burn_llc/startube_div': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'brown': 'barnett'}, inplace = True)
cleaning6['make'].replace({'eurocopter_deutsch': 'eurocopter'}, inplace = True)
cleaning6['make'].replace({'sundt_daniel_n': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'randall_amanda': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'cotton_galen_m': 'eagle'}, inplace = True)
cleaning6['make'].replace({'sands_thomas_e': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'buschmann_robert_e': 'eagle'}, inplace = True)
cleaning6['make'].replace({'holland': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'schultz': 'revolution'}, inplace = True)
cleaning6['make'].replace({'vic_tatelman': 'safari'}, inplace = True)
cleaning6['make'].replace({'dowell_vaughn': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'willey': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'brooks': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'rogerson_hiller_corp.': 'hiller'}, inplace = True)
cleaning6['make'].replace({'eagle_rnd': 'eagle'}, inplace = True)
cleaning6['make'].replace({'rallison': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'wilkinson_daniel_v': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'mccall': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'boykin': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'calstar_1': 'bolkow'}, inplace = True)
cleaning6['make'].replace({'belcher': 'eagle'}, inplace = True)
cleaning6['make'].replace({'lawing_thomas_r': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'md_helicopters_inc.': 'md_helicopter'}, inplace = True)
cleaning6['make'].replace({'campbell_w/smiley_j': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'mcfarland': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'helo_werks': 'helowerks'}, inplace = True)
cleaning6['make'].replace({'palulis_john': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'kearsley': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'deussen_alexander_ii': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'west': 'revolution'}, inplace = True)
cleaning6['make'].replace({'messerschmitt': 'bolkow'}, inplace = True)
cleaning6['make'].replace({'callahan': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'williams_helicopter': 'bell'}, inplace = True)
cleaning6['make'].replace({'garlick_helicopters_inc.': 'bell'}, inplace = True)
cleaning6['make'].replace({'buss': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'tjerrild': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'steward_rocky': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'shapowal': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'gray': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'boucher': 'revolution'}, inplace = True)
cleaning6['make'].replace({'zhuhai_helicopter_co.': 'sikorsky'}, inplace = True)
cleaning6['make'].replace({'southern_aero': 'bell'}, inplace = True)
cleaning6['make'].replace({'vanhoose/scorpio': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'airbus_helicopter': 'eurocopter'}, inplace = True)
cleaning6['make'].replace({'tomcat': 'continental'}, inplace = True)
cleaning6['make'].replace({'rotorway_aircraft,_inc.': 'rotorway'}, inplace = True)
cleaning6['make'].replace({'continental_mk5a': 'bell'}, inplace = True)
cleaning6['make'].replace({'bell_47g-3b': 'bell'}, inplace = True)
cleaning6['make'].replace({'moore_bell': 'bell'}, inplace = True)
cleaning6['make'].replace({'garlick': 'bell'}, inplace = True)
cleaning6['make'].replace({'kawasaki': 'bolkow'}, inplace = True)
cleaning6['make'].replace({'kawaski': 'bolkow'}, inplace = True)
cleaning6['make'].replace({'michael_r_alexander': 'mosquito'}, inplace = True)
cleaning6['make'].replace({'heard_alan_e': 'mosquito'}, inplace = True)
cleaning6['make'].replace({'brad_j_bates': 'mosquito'}, inplace = True)
cleaning6['make'].replace({'d_&_k_aviation': 'chr'}, inplace = True)
cleaning6['make'].replace({'mbb-bolkow': 'bolkow'}, inplace = True)
cleaning6['make'].replace({'boeing-vertol': 'boeing'}, inplace = True)
cleaning6['make'].replace({'brantley': 'brantly'}, inplace = True)
cleaning6['make'].replace({'westland': 'agusta'}, inplace = True)
cleaning6['make'].replace({'pzl_swidnik': 'pzl'}, inplace = True)

In [1748]:
# I note some unfamiliar helicopter makes when looking at the single values.
cleaning6['make'].value_counts()

bell                   819
robinson               739
hughes                 220
eurocopter             204
schweizer              103
md_helicopter          92 
aerospatiale           86 
rotorway               73 
enstrom                72 
sikorsky               68 
hiller                 64 
agusta                 35 
bolkow                 25 
brantly                22 
eagle                  11 
mosquito               6  
guimbal                5  
kaman                  5  
continental            4  
revolution             4  
safari                 4  
nan                    3  
boeing                 3  
hummingbird            2  
pzl                    2  
hal                    2  
moore_david            1  
sontonastaso           1  
johnston_douglas_s     1  
schulman               1  
guisinger_gary_g       1  
sud_aviation           1  
piasecki               1  
chapel_robert_b        1  
michael_s/michael_k    1  
chr                    1  
bender_joseph          1  
b

In [1749]:
# note that when the make is nan, the models are amateur
# (rotorway is amateur built despite the nan in the amateur_built column) https://en.wikipedia.org/wiki/RotorWay
cleaning6[cleaning6['make']== 'nan']

,investigation_type,event_date,location,country,latitude,longitude,make,model,broad_phase_of_flight,amateur_built,number_of_engines,engine_type,far_description,purpose_of_flight,injury_severity,aircraft_damage,weather_condition,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,people_on_board,registration_number
1255,accident,02/18/2012,"Lahr, Germany",Germany,NaN,NaN,nan,safari 400,NaN,Yes,1,NaN,"Non-U.S., Non-Commercial",Personal,fatal(1),Destroyed,VMC,1,0,0,0,1,NaN
1820,accident,11/16/2008,"Lingen, Germany",Germany,52.455556,7.184722,nan,rotorway,NaN,NaN,1,NaN,"Non-U.S., Non-Commercial",Unknown,fatal(1),Substantial,VMC,1,0,0,0,1,NaN
2442,accident,10/23/1982,"ELOY, AZ",United States,NaN,NaN,nan,hobbs b8m,APPROACH,Yes,1,Reciprocating,Part 91: General Aviation,Personal,fatal(1),Substantial,VMC,1,0,0,0,1,N1335G


In [1750]:
cleaning6['make'].replace({'nan': 'rotorway'}, inplace = True)

In [1751]:
cleaning6[cleaning6['make']=='nan']

,investigation_type,event_date,location,country,latitude,longitude,make,model,broad_phase_of_flight,amateur_built,number_of_engines,engine_type,far_description,purpose_of_flight,injury_severity,aircraft_damage,weather_condition,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,people_on_board,registration_number


##  

## Amateur built

In [1752]:
cleaning6.amateur_built.value_counts()

 No      2535
 Yes     128 
Name: amateur_built, dtype: int64

In [1753]:
cleaning6.amateur_built=[str(i).strip().lower() for i in cleaning6.amateur_built]

In [1754]:
# code so I can see all rows to visually check make names against amateur built column
pd.set_option('display.max_columns', 500)


In [1755]:
cleaning6[cleaning6['amateur_built']== 'yes']

,investigation_type,event_date,location,country,latitude,longitude,make,model,broad_phase_of_flight,amateur_built,number_of_engines,engine_type,far_description,purpose_of_flight,injury_severity,aircraft_damage,weather_condition,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,people_on_board,registration_number
52,accident,10/27/2018,"Passaic, MO",United States,38.321945,-94.346944,rotorway,162,MANEUVERING,yes,1,Reciprocating,Part 91: General Aviation,Personal,non-fatal,Substantial,VMC,0,0,0,1,1,N162RF
69,accident,09/12/2018,"Billings, MT",United States,45.000000,-108.000000,robert_l_cooons,rw 1,MANEUVERING,yes,1,Turbo Shaft,Part 91: General Aviation,Personal,fatal(1),Substantial,VMC,1,0,0,0,1,N166LC
71,accident,09/08/2018,"Highgate, VT",United States,44.933611,-73.094444,safari,safari,NaN,yes,1,Reciprocating,Part 91: General Aviation,Instructional,non-fatal,Substantial,VMC,0,0,0,1,1,N699WC
119,accident,07/02/2018,"Aguilares, TX",United States,27.150000,-99.110000,safari,400,NaN,yes,1,Unknown,Part 91: General Aviation,Personal,non-fatal,Substantial,VMC,0,1,0,0,1,N326RW
123,accident,06/25/2018,"Beach City, OH",United States,40.646389,-81.555833,sibila,mh-1,TAKEOFF,yes,1,Reciprocating,Part 91: General Aviation,Personal,non-fatal,Substantial,VMC,0,1,0,0,1,N32KS
147,accident,05/02/2018,"Iron Station, NC",United States,35.440000,-81.120000,mosquito,mosquito xe,MANEUVERING,yes,1,Reciprocating,Part 91: General Aviation,Personal,non-fatal,Destroyed,VMC,0,0,1,0,1,N725JC
155,accident,04/24/2018,"Albion, NY",United States,43.296389,-78.222500,mosquito,mosquito xe,UNKNOWN,yes,1,Reciprocating,Part 91: General Aviation,Personal,non-fatal,Destroyed,VMC,0,1,0,0,1,N911CY
164,accident,04/05/2018,"Crestet, France",France,44.214444,5.105000,rotorway,exec,CRUISE,yes,1,Reciprocating,"Non-U.S., Non-Commercial",NaN,fatal(1),Destroyed,NaN,1,0,0,0,1,NaN
167,accident,03/25/2018,"Koblenz-Winningen, Germany",Germany,50.325000,7.533056,rotorway,exec,UNKNOWN,yes,1,NaN,Unknown,NaN,non-fatal,Substantial,NaN,0,0,0,2,2,NaN
195,accident,01/28/2018,"Harrisville, NY",United States,44.297777,-75.246944,hummingbird,260l hummingbird,NaN,yes,1,Reciprocating,Part 91: General Aviation,Personal,non-fatal,Substantial,VMC,0,0,0,1,1,N4045


In [1756]:
# I note some incorrectly classified helicopters (yes in amateur built when it should be no)
cleaning6.loc[cleaning6['make'].str.contains('eurocopter'), 'amateur_built']= 'no'
cleaning6.loc[cleaning6['make'].str.contains('bell'), 'amateur_built']= 'no'
cleaning6.loc[cleaning6['make'].str.contains('sikorsky'), 'amateur_built']= 'no'
cleaning6.loc[cleaning6['make'].str.contains('hughes'), 'amateur_built']= 'no'
cleaning6.loc[cleaning6['make'].str.contains('schweizer'), 'amateur_built']= 'no'

In [1757]:
# decision that as there are so few amateur built helicopters, and in some cases, the make names look to be the name of the
# pilot who built the helicopter in some cases, to group them together as 'amateur'

In [1758]:
cleaning6.loc[cleaning6['amateur_built'].str.contains('yes'), 'make']= 'amateur'  

In [1759]:
# also need to check if any makes were misclassified as being professionally built when they were amateur built.
# There are three in the list below that need to move to 'amateur'
cleaning6.make.value_counts()

bell                819
robinson            739
hughes              220
eurocopter          204
amateur             119
schweizer           103
md_helicopter       92 
aerospatiale        86 
enstrom             72 
sikorsky            68 
hiller              64 
agusta              35 
bolkow              25 
brantly             22 
rotorway            6  
kaman               5  
guimbal             5  
continental         4  
boeing              3  
pzl                 2  
hal                 2  
aero_eli_servizi    1  
boeing_vertol       1  
helowerks           1  
embraer             1  
sud_aviation        1  
mosquito            1  
mil                 1  
Name: make, dtype: int64

In [1760]:
# first changing amateur built column to yes, for those that are amateur built
cleaning6.loc[cleaning6['make'].str.contains('rotorway'), 'amateur_built']= 'yes'
cleaning6.loc[cleaning6['make'].str.contains('helowerks'), 'amateur_built']= 'yes'
cleaning6.loc[cleaning6['make'].str.contains('mosquito'), 'amateur_built']= 'yes'

In [1761]:
# now changing the makes to amateur as well
cleaning6['make'].replace({'rotorway': 'amateur'}, inplace = True)
cleaning6['make'].replace({'helowerks': 'amateur'}, inplace = True)
cleaning6['make'].replace({'mosquito': 'amateur'}, inplace = True)

In [1762]:
# will also do this with model
cleaning6.loc[cleaning6['amateur_built'].str.contains('yes'), 'model']= 'amateur' 

In [1763]:
cleaning6[cleaning6['amateur_built']== 'yes']

,investigation_type,event_date,location,country,latitude,longitude,make,model,broad_phase_of_flight,amateur_built,number_of_engines,engine_type,far_description,purpose_of_flight,injury_severity,aircraft_damage,weather_condition,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,people_on_board,registration_number
43,accident,11/05/2018,"Saint Pierre, France, France",France,46.830556,56.280556,amateur,amateur,NaN,yes,NaN,NaN,Unknown,NaN,fatal(1),Destroyed,NaN,1,1,0,0,2,NaN
52,accident,10/27/2018,"Passaic, MO",United States,38.321945,-94.346944,amateur,amateur,MANEUVERING,yes,1,Reciprocating,Part 91: General Aviation,Personal,non-fatal,Substantial,VMC,0,0,0,1,1,N162RF
69,accident,09/12/2018,"Billings, MT",United States,45.000000,-108.000000,amateur,amateur,MANEUVERING,yes,1,Turbo Shaft,Part 91: General Aviation,Personal,fatal(1),Substantial,VMC,1,0,0,0,1,N166LC
71,accident,09/08/2018,"Highgate, VT",United States,44.933611,-73.094444,amateur,amateur,NaN,yes,1,Reciprocating,Part 91: General Aviation,Instructional,non-fatal,Substantial,VMC,0,0,0,1,1,N699WC
119,accident,07/02/2018,"Aguilares, TX",United States,27.150000,-99.110000,amateur,amateur,NaN,yes,1,Unknown,Part 91: General Aviation,Personal,non-fatal,Substantial,VMC,0,1,0,0,1,N326RW
123,accident,06/25/2018,"Beach City, OH",United States,40.646389,-81.555833,amateur,amateur,TAKEOFF,yes,1,Reciprocating,Part 91: General Aviation,Personal,non-fatal,Substantial,VMC,0,1,0,0,1,N32KS
147,accident,05/02/2018,"Iron Station, NC",United States,35.440000,-81.120000,amateur,amateur,MANEUVERING,yes,1,Reciprocating,Part 91: General Aviation,Personal,non-fatal,Destroyed,VMC,0,0,1,0,1,N725JC
155,accident,04/24/2018,"Albion, NY",United States,43.296389,-78.222500,amateur,amateur,UNKNOWN,yes,1,Reciprocating,Part 91: General Aviation,Personal,non-fatal,Destroyed,VMC,0,1,0,0,1,N911CY
164,accident,04/05/2018,"Crestet, France",France,44.214444,5.105000,amateur,amateur,CRUISE,yes,1,Reciprocating,"Non-U.S., Non-Commercial",NaN,fatal(1),Destroyed,NaN,1,0,0,0,1,NaN
167,accident,03/25/2018,"Koblenz-Winningen, Germany",Germany,50.325000,7.533056,amateur,amateur,UNKNOWN,yes,1,NaN,Unknown,NaN,non-fatal,Substantial,NaN,0,0,0,2,2,NaN


In [1764]:
cleaning7=cleaning6.copy()

##  

## Model

In [1765]:
#  Putting an underscore into blank spaces in the model name
cleaning7.model=[i.replace(' ','_') for i in cleaning7.model]

In [1766]:
# Standardising the model names
cleaning7['model'].replace({'r22_beta': 'r22'}, inplace = True)
cleaning7['model'].replace({'r44_ii': 'r44'}, inplace = True)
cleaning7['model'].replace({'r-22': 'r22'}, inplace = True)
cleaning7['model'].replace({'r22_beta_ii': 'r22'}, inplace = True)
cleaning7['model'].replace({'r-44': 'r44'}, inplace = True)
cleaning7['model'].replace({'r44_raven_i': 'r44'}, inplace = True)
cleaning7['model'].replace({'bk_117_a-3': 'bk117'}, inplace = True)
cleaning7['model'].replace({'r-22_hp': 'r22'}, inplace = True)
cleaning7['model'].replace({'ec-145': 'ec145'}, inplace = True)
cleaning7['model'].replace({'as_350_b3': 'as350'}, inplace = True)
cleaning7['model'].replace({'as_350_b2': 'as350'}, inplace = True)
cleaning7['model'].replace({'as350b2': 'as350'}, inplace = True)
cleaning7['model'].replace({'s_350_b3': 'as350'}, inplace = True)
cleaning7['model'].replace({'as_350_ba_fx1': 'as350'}, inplace = True)
cleaning7['model'].replace({'r-44-ii': 'r44'}, inplace = True)
cleaning7['model'].replace({'ec_130_b4': 'ec130'}, inplace = True)
cleaning7['model'].replace({'206l-1': '206'}, inplace = True)
cleaning7['model'].replace({'as_350_b3': 'as350'}, inplace = True)
cleaning7['model'].replace({'206l-3': '206'}, inplace = True)
cleaning7['model'].replace({'r_22_beta': 'r22'}, inplace = True)
cleaning7['model'].replace({'a350': 'as350'}, inplace = True)
cleaning7['model'].replace({'bk-117-a4': 'bk117'}, inplace = True)
cleaning7['model'].replace({'r_22_beta': 'r22'}, inplace = True)
cleaning7['model'].replace({'r44_-_ii': 'r44'}, inplace = True)
cleaning7['model'].replace({'as350b3_2b1': 'as350'}, inplace = True)
cleaning7['model'].replace({'r44_-_ii': 'r44'}, inplace = True)
cleaning7['model'].replace({'as350b3': 'as350'}, inplace = True)
cleaning7['model'].replace({'206b': '206'}, inplace = True)
cleaning7['model'].replace({'369d': '369'}, inplace = True)
cleaning7['model'].replace({'369ff': '369'}, inplace = True)
cleaning7['model'].replace({'269c-1': '269'}, inplace = True)
cleaning7['model'].replace({'269a': '269'}, inplace = True)
cleaning7['model'].replace({'369e': '369'}, inplace = True)
cleaning7['model'].replace({'369a': '369'}, inplace = True)
cleaning7['model'].replace({'206b_ii': '206'}, inplace = True)
cleaning7['model'].replace({'a109e': 'a109'}, inplace = True)
cleaning7['model'].replace({'269b': '269'}, inplace = True)
cleaning7['model'].replace({'206_-_l3': '206'}, inplace = True)
cleaning7['model'].replace({'269c': '269'}, inplace = True)
cleaning7['model'].replace({'206l3': '206'}, inplace = True)
cleaning7['model'].replace({'ec135t2+': 'ec135'}, inplace = True)
cleaning7['model'].replace({'206_-_l3': '206'}, inplace = True)
cleaning7['model'].replace({'as350ba ': 'as350'}, inplace = True)
cleaning7['model'].replace({'mbb-bk_117_c-1': 'bk117'}, inplace = True)
cleaning7['model'].replace({'uh-1h ': 'uh1'}, inplace = True)
cleaning7['model'].replace({'uh-1b': 'uh1'}, inplace = True)
cleaning7['model'].replace({'uh-1h': 'uh1'}, inplace = True)
cleaning7['model'].replace({'uh-12e': 'uh12'}, inplace = True)
cleaning7['model'].replace({'uh-12c': 'uh12'}, inplace = True)
cleaning7['model'].replace({'uh_1h': 'uh1'}, inplace = True)
cleaning7['model'].replace({'47g-3b-1': '47'}, inplace = True)
cleaning7['model'].replace({'47g-2a': '47'}, inplace = True)
cleaning7['model'].replace({'47g-2': '47'}, inplace = True)
cleaning7['model'].replace({'47g-4a': '47'}, inplace = True)
cleaning7['model'].replace({'as350ba': 'as350'}, inplace = True)
cleaning7['model'].replace({'206l': '206'}, inplace = True)
cleaning7['model'].replace({'b206l-1': '206'}, inplace = True)
cleaning7['model'].replace({'47g-3b-l': '47'}, inplace = True)
cleaning7['model'].replace({'s76a': 's76'}, inplace = True)
cleaning7['model'].replace({'oh-58a': 'oh58'}, inplace = True)
cleaning7['model'].replace({'oh-58c': 'oh58'}, inplace = True)
cleaning7['model'].replace({'ec-135': 'ec135'}, inplace = True)
cleaning7['model'].replace({'47g4': '47'}, inplace = True)
cleaning7['model'].replace({'47g4': '47'}, inplace = True)
cleaning7['model'].replace({'47d1': '47'}, inplace = True)
cleaning7['model'].replace({'47g': '47'}, inplace = True)
cleaning7['model'].replace({'oh_58a': 'oh58'}, inplace = True)
cleaning7['model'].replace({'as350-b2': 'as350'}, inplace = True)
cleaning7['model'].replace({'bo-105c': '105'}, inplace = True)
cleaning7['model'].replace({'r22_-_beta': 'r22'}, inplace = True)
cleaning7['model'].replace({'206-l4': '206'}, inplace = True)
cleaning7['model'].replace({'f-28c': 'f28'}, inplace = True)
cleaning7['model'].replace({'f-28a': 'f28'}, inplace = True)
cleaning7['model'].replace({'as_350b3': 'as350'}, inplace = True)
cleaning7['model'].replace({'206_b_iii': '206'}, inplace = True)
cleaning7['model'].replace({'as350_b3e': 'as350'}, inplace = True)
cleaning7['model'].replace({'47_g': '47'}, inplace = True)
cleaning7['model'].replace({'47k': '47'}, inplace = True)
cleaning7['model'].replace({'47-g5': '47'}, inplace = True)
cleaning7['model'].replace({'47g_4a': '47'}, inplace = True)
cleaning7['model'].replace({'47-b3': '47'}, inplace = True)
cleaning7['model'].replace({'h-369hs': '369'}, inplace = True)
cleaning7['model'].replace({'407t': '407'}, inplace = True)
cleaning7['model'].replace({'ec_135_p2+': 'ec135'}, inplace = True)
cleaning7['model'].replace({'ec_135_p2': 'ec135'}, inplace = True)
cleaning7['model'].replace({'as350_b2_e': 'as350'}, inplace = True)
cleaning7['model'].replace({'b-206l-1': '206'}, inplace = True)
cleaning7['model'].replace({'b206_l-3': '206'}, inplace = True)
cleaning7['model'].replace({'b-206l-1': '206'}, inplace = True)
cleaning7['model'].replace({'bell_206b': '206'}, inplace = True)
cleaning7['model'].replace({'ec_135_p2': 'ec135'}, inplace = True)
cleaning7['model'].replace({'ab206b': '206'}, inplace = True)
cleaning7['model'].replace({'ec_135_t2+': 'ec135'}, inplace = True)
cleaning7['model'].replace({'ec_135_t2+': 'ec135'}, inplace = True)
cleaning7['model'].replace({'as_355': 'as355'}, inplace = True)
cleaning7['model'].replace({'ec130_b4': 'ec130'}, inplace = True)
cleaning7['model'].replace({'r-22_beta_2': 'r22'}, inplace = True)
cleaning7['model'].replace({'f28_-_c': 'f28'}, inplace = True)
cleaning7['model'].replace({'bk-117_b-2': 'bk117'}, inplace = True)
cleaning7['model'].replace({'280-c': '280'}, inplace = True)
cleaning7['model'].replace({'280f': '280'}, inplace = True)
cleaning7['model'].replace({'c-130-b4': 'ec130'}, inplace = True)
cleaning7['model'].replace({'280fx': '280'}, inplace = True)
cleaning7['model'].replace({'as_350': 'as350'}, inplace = True)
cleaning7['model'].replace({'a-109-k2': 'a109'}, inplace = True)
cleaning7['model'].replace({'47g-1': '47'}, inplace = True)
cleaning7['model'].replace({'ec-130-b4': 'ec130'}, inplace = True)
cleaning7['model'].replace({'md-600': 'md600'}, inplace = True)
cleaning7['model'].replace({'md_500e': 'md500'}, inplace = True)
cleaning7['model'].replace({'s-76c++': 's76'}, inplace = True)
cleaning7['model'].replace({'uh-60m_(s-70)': 'uh60'}, inplace = True)
cleaning7['model'].replace({'s-76c++': 's76'}, inplace = True)
cleaning7['model'].replace({'f28f': 'f28'}, inplace = True)
cleaning7['model'].replace({'as_355f2': 'as355'}, inplace = True)
cleaning7['model'].replace({'269c_-_1': '269'}, inplace = True)
cleaning7['model'].replace({'205a1': '205'}, inplace = True)
cleaning7['model'].replace({'bell_47': '47'}, inplace = True)
cleaning7['model'].replace({'206-l': '206'}, inplace = True)
cleaning7['model'].replace({'uh_12e': 'uh12'}, inplace = True)
cleaning7['model'].replace({'206b-3': '206'}, inplace = True)
cleaning7['model'].replace({'as-350_ba': 'as350'}, inplace = True)
cleaning7['model'].replace({'s-76a++': 's76'}, inplace = True)
cleaning7['model'].replace({'se3130_alouette_ii': 'alouette'}, inplace = True)
cleaning7['model'].replace({'el_tomcat-mk5a': '47'}, inplace = True)
cleaning7['model'].replace({'el_tomcat_mk5a': '47'}, inplace = True)
cleaning7['model'].replace({'s76-c++': 's76'}, inplace = True)
cleaning7['model'].replace({'350b1': 'as350'}, inplace = True)
cleaning7['model'].replace({'350d': 'as350'}, inplace = True)
cleaning7['model'].replace({'47g3b': '47'}, inplace = True)
cleaning7['model'].replace({'as-355f1': 'as355'}, inplace = True)
cleaning7['model'].replace({'f28c-2': 'f28'}, inplace = True)
cleaning7['model'].replace({'oh58': 'oh_58'}, inplace = True)
cleaning7['model'].replace({'oh58a': 'oh_58'}, inplace = True)
cleaning7['model'].replace({'as350-d ': 'as350'}, inplace = True)
cleaning7['model'].replace({'as350ba_(fx2)': 'as350'}, inplace = True)
cleaning7['model'].replace({'s_76b': 's76'}, inplace = True)
cleaning7['model'].replace({'47g-2a1': '47'}, inplace = True)
cleaning7['model'].replace({'bk117a3': 'bk117'}, inplace = True)
cleaning7['model'].replace({'269_c-1': '269'}, inplace = True)
cleaning7['model'].replace({'as_365_n-2': 'as365'}, inplace = True)
cleaning7['model'].replace({'bk-117-b2': 'bk117'}, inplace = True)
cleaning7['model'].replace({'as-350b_ecureuil': 'as350'}, inplace = True)
cleaning7['model'].replace({'r44_-_undesignat': 'r44'}, inplace = True)
cleaning7['model'].replace({'214b': '214'}, inplace = True)
cleaning7['model'].replace({'r-22hp': 'r22'}, inplace = True)
cleaning7['model'].replace({'as350b1': 'as350'}, inplace = True)
cleaning7['model'].replace({'407_gx': '407'}, inplace = True)
cleaning7['model'].replace({'500-e': 'md500'}, inplace = True)
cleaning7['model'].replace({'as_350_b-2': 'as350'}, inplace = True)
cleaning7['model'].replace({'bk117-c1': 'bk117'}, inplace = True)
cleaning7['model'].replace({'as350-d': 'as350'}, inplace = True)
cleaning7['model'].replace({'aw109sp': 'a109'}, inplace = True)
cleaning7['model'].replace({'206_b2': '206'}, inplace = True)
cleaning7['model'].replace({'206biii': '206'}, inplace = True)
cleaning7['model'].replace({'ec_130_t2': 'ec130'}, inplace = True)
cleaning7['model'].replace({'bk117c1': 'bk117'}, inplace = True)
cleaning7['model'].replace({'as-355-f2': 'as355'}, inplace = True)
cleaning7['model'].replace({'as350d_astar': 'as350'}, inplace = True)
cleaning7['model'].replace({'47g_super_c_4': '47'}, inplace = True)
cleaning7['model'].replace({'47g2a1': '47'}, inplace = True)
cleaning7['model'].replace({'bo-105cbs': '105'}, inplace = True)
cleaning7['model'].replace({'bk117_b-2': 'bk117'}, inplace = True)
cleaning7['model'].replace({'ab206-b3': '206'}, inplace = True)
cleaning7['model'].replace({'bo_105-s': '105'}, inplace = True)
cleaning7['model'].replace({'bo_105ls_a-3': '105'}, inplace = True)
cleaning7['model'].replace({'aw-139': '139'}, inplace = True)
cleaning7['model'].replace({'500c': '500'}, inplace = True)
cleaning7['model'].replace({'269-c': '269'}, inplace = True)
cleaning7['model'].replace({'369hs': '369'}, inplace = True)
cleaning7['model'].replace({'th_55': '500'}, inplace = True)
cleaning7['model'].replace({'oh_58': '206'}, inplace = True)
cleaning7['model'].replace({'uh1b': '212'}, inplace = True)
cleaning7['model'].replace({'hh-60l': 'seaking'}, inplace = True)
cleaning7['model'].replace({'47g': '47'}, inplace = True)
cleaning7['model'].replace({'206b_iii': '206'}, inplace = True)
cleaning7['model'].replace({'47g2': '47'}, inplace = True)
cleaning7['model'].replace({'206_l-1': '206'}, inplace = True)
cleaning7['model'].replace({'wcs-222_(bell_47g)': '47'}, inplace = True)
cleaning7['model'].replace({'280c': '280'}, inplace = True)
cleaning7['model'].replace({'214b-1': '214'}, inplace = True)
cleaning7['model'].replace({'sa-316b': '316'}, inplace = True)
cleaning7['model'].replace({'sa-360c': '360'}, inplace = True)
cleaning7['model'].replace({'hh-43f': 'hh43'}, inplace = True)
cleaning7['model'].replace({'uh_12': 'uh12'}, inplace = True)
cleaning7['model'].replace({'uh-19b': 'uh19'}, inplace = True)
cleaning7['model'].replace({'s-58j': 'h34'}, inplace = True)
cleaning7['model'].replace({'oh_58c': 'oh58'}, inplace = True)
cleaning7['model'].replace({'agusta_aw169': '169'}, inplace = True)
cleaning7['model'].replace({'uh_-1h': '212'}, inplace = True)
cleaning7['model'].replace({'md-369': '369'}, inplace = True)
cleaning7['model'].replace({'412ep': '412'}, inplace = True)
cleaning7['model'].replace({'47g_3b': '47'}, inplace = True)
cleaning7['model'].replace({'b-2': 'b2'}, inplace = True)
cleaning7['model'].replace({'269c1': '269'}, inplace = True)
cleaning7['model'].replace({'tomcat_mk5a': '47'}, inplace = True)
cleaning7['model'].replace({'47j': '47'}, inplace = True)
cleaning7['model'].replace({'269d': '269'}, inplace = True)
cleaning7['model'].replace({'ec135p1': 'ec135'}, inplace = True)
cleaning7['model'].replace({'as350_b2_ecureuil': 'as350'}, inplace = True)
cleaning7['model'].replace({'206l-4': '206'}, inplace = True)
cleaning7['model'].replace({'bk_117_b-2': 'bk117'}, inplace = True)
cleaning7['model'].replace({'s-92a': 's92'}, inplace = True)
cleaning7['model'].replace({'sa330j_puma': 'puma'}, inplace = True)
cleaning7['model'].replace({'cabri_g2': 'cabri'}, inplace = True)
cleaning7['model'].replace({'ec_130': 'ec130'}, inplace = True)
cleaning7['model'].replace({'369hm': '369'}, inplace = True)
cleaning7['model'].replace({'412sp': '412'}, inplace = True)
cleaning7['model'].replace({'b_2b': 'b2'}, inplace = True)
cleaning7['model'].replace({'47g_5': '47'}, inplace = True)
cleaning7['model'].replace({'mbb_bk_117': 'bk117'}, inplace = True)
cleaning7['model'].replace({'206l_3': '206'}, inplace = True)
cleaning7['model'].replace({'uh_12a': 'uh12'}, inplace = True)
cleaning7['model'].replace({'aw139': '139'}, inplace = True)
cleaning7['model'].replace({'f_28f': 'f28'}, inplace = True)
cleaning7['model'].replace({'s-97a': 's97'}, inplace = True)
cleaning7['model'].replace({'f_28c': 'f28'}, inplace = True)
cleaning7['model'].replace({'schweizer_269c': '269'}, inplace = True)
cleaning7['model'].replace({'bk_117_b2': 'bk117'}, inplace = True)
cleaning7['model'].replace({'ec_135': 'ec135'}, inplace = True)
cleaning7['model'].replace({'oh-58a+': 'oh58'}, inplace = True)
cleaning7['model'].replace({'mbb_bk_117_c-2': 'bk117'}, inplace = True)
cleaning7['model'].replace({'oh_23b': 'oh23'}, inplace = True)
cleaning7['model'].replace({'oh_6a': 'oh6'}, inplace = True)
cleaning7['model'].replace({'robinson_r44_ii': 'r44'}, inplace = True)
cleaning7['model'].replace({'k_1200': 'k1200'}, inplace = True)
cleaning7['model'].replace({'a109a_ii': 'a109'}, inplace = True)
cleaning7['model'].replace({'206_l4': '206'}, inplace = True)
cleaning7['model'].replace({'uh_12d': 'uh12'}, inplace = True)
cleaning7['model'].replace({'269c_1': '269'}, inplace = True)
cleaning7['model'].replace({'47g_2': '47'}, inplace = True)
cleaning7['model'].replace({'oh_13h/m74a': 'oh13'}, inplace = True)
cleaning7['model'].replace({'47g_3b_1': '47'}, inplace = True)
cleaning7['model'].replace({'uh_1b': 'uh1'}, inplace = True)
cleaning7['model'].replace({'th-180': 'th180'}, inplace = True)
cleaning7['model'].replace({'600n': '600'}, inplace = True)
cleaning7['model'].replace({'md600': '600'}, inplace = True)
cleaning7['model'].replace({'f-28': 'f28'}, inplace = True)
cleaning7['model'].replace({'as350b3e': 'as350'}, inplace = True)
cleaning7['model'].replace({'ec120b': 'ec120'}, inplace = True)
cleaning7['model'].replace({'269c_-_undesignat': '269'}, inplace = True)
cleaning7['model'].replace({'s_61a': 's61'}, inplace = True)
cleaning7['model'].replace({'s76_-_c': 's76'}, inplace = True)
cleaning7['model'].replace({'206_-_l1': '206'}, inplace = True)
cleaning7['model'].replace({'407_-_no_series': '407'}, inplace = True)
cleaning7['model'].replace({'r-22_beta_ii': 'r22'}, inplace = True)
cleaning7['model'].replace({'206_-_b11': '206'}, inplace = True)
cleaning7['model'].replace({'206_-_l4': '206'}, inplace = True)
cleaning7['model'].replace({'206_-_b': '206'}, inplace = True)
cleaning7['model'].replace({'as-350': 'as350'}, inplace = True)
cleaning7['model'].replace({'as350_b2': 'as350'}, inplace = True)
cleaning7['model'].replace({'47g_2a': '47'}, inplace = True)
cleaning7['model'].replace({'as350_b2': 'as350'}, inplace = True)
cleaning7['model'].replace({'aw119_mkii': 'aw119'}, inplace = True)
cleaning7['model'].replace({'as350b': 'as350'}, inplace = True)
cleaning7['model'].replace({'as_350_b2_ecureuil': 'as350'}, inplace = True)
cleaning7['model'].replace({'md_900': '900'}, inplace = True)
cleaning7['model'].replace({'206l_1': '206'}, inplace = True)
cleaning7['model'].replace({'500n': '500'}, inplace = True)
cleaning7['model'].replace({'206a': '206'}, inplace = True)
cleaning7['model'].replace({'as-350b2': 'as350'}, inplace = True)
cleaning7['model'].replace({'oh_58a+': 'oh58'}, inplace = True)
cleaning7['model'].replace({'as_365_n2_dauphin': 'as365'}, inplace = True)
cleaning7['model'].replace({'th_55a': '500'}, inplace = True)
cleaning7['model'].replace({'bk_117': 'bk117'}, inplace = True)
cleaning7['model'].replace({'oh_6': 'oh6'}, inplace = True)
cleaning7['model'].replace({'280_-_fx': '289fx'}, inplace = True)
cleaning7['model'].replace({'agusta_aw139': 'aw139'}, inplace = True)
cleaning7['model'].replace({'47g_5a': '47'}, inplace = True)
cleaning7['model'].replace({'f_28a': 'f28'}, inplace = True)
cleaning7['model'].replace({'r_44': 'r44'}, inplace = True)
cleaning7['model'].replace({'r22_-_alpha': 'r22'}, inplace = True)
cleaning7['model'].replace({'bk117-b2': 'bk117'}, inplace = True)
cleaning7['model'].replace({'47-g3b2': '47'}, inplace = True)
cleaning7['model'].replace({'as-350-d': 'as350'}, inplace = True)
cleaning7['model'].replace({'b-2b': 'b2'}, inplace = True)
cleaning7['model'].replace({'s76_-_a': 's76'}, inplace = True)
cleaning7['model'].replace({'ec_135_t2': 'ec135'}, inplace = True)
cleaning7['model'].replace({'206_b': '206'}, inplace = True)
cleaning7['model'].replace({'bk-117_a-4': 'bk117'}, inplace = True)
cleaning7['model'].replace({'206_b_ii': '206'}, inplace = True)
cleaning7['model'].replace({'uh-1f': 'uh1'}, inplace = True)
cleaning7['model'].replace({'ec_130b4': 'ec130'}, inplace = True)
cleaning7['model'].replace({'alouette_ii_se313b': 'alouette'}, inplace = True)
cleaning7['model'].replace({'369d_(500d)': '369'}, inplace = True)
cleaning7['model'].replace({'r-22a': 'r22'}, inplace = True)
cleaning7['model'].replace({'369f': '369'}, inplace = True)
cleaning7['model'].replace({'206-l1': '206'}, inplace = True)
cleaning7['model'].replace({'as365n3': 'as365'}, inplace = True)
cleaning7['model'].replace({'206-b2': '206'}, inplace = True)
cleaning7['model'].replace({'uh12d': 'uh12'}, inplace = True)
cleaning7['model'].replace({'uh-12l3': 'uh12'}, inplace = True)
cleaning7['model'].replace({'beta_ii': 'r22'}, inplace = True)
cleaning7['model'].replace({'369e_(md500e)': '369'}, inplace = True)
cleaning7['model'].replace({'369b': '369'}, inplace = True)
cleaning7['model'].replace({'f-28c-2': 'f28'}, inplace = True)
cleaning7['model'].replace({'robinson_r44': 'r44'}, inplace = True)
cleaning7['model'].replace({'m74l': 'oh13'}, inplace = True)
cleaning7['model'].replace({'212_-_no_series': '212'}, inplace = True)
cleaning7['model'].replace({'oh-13e/m74': 'oh13'}, inplace = True)
cleaning7['model'].replace({'47g5': '47'}, inplace = True)
cleaning7['model'].replace({'uh-1v': 'uh1'}, inplace = True)
cleaning7['model'].replace({'332l2': 'as332'}, inplace = True)
cleaning7['model'].replace({'r22_mariner': 'r22'}, inplace = True)
cleaning7['model'].replace({'oh-13h/m74a': 'oh13'}, inplace = True)
cleaning7['model'].replace({'uh-12a': 'uh12'}, inplace = True)
cleaning7['model'].replace({'th-55a': '500'}, inplace = True)
cleaning7['model'].replace({'76a': 's76'}, inplace = True)
cleaning7['model'].replace({'s-76c': 's76'}, inplace = True)
cleaning7['model'].replace({'s-64e': 's64'}, inplace = True)
cleaning7['model'].replace({'sa_318c': 'sa318'}, inplace = True)
cleaning7['model'].replace({'bht407': '407'}, inplace = True)
cleaning7['model'].replace({'f-28f': 'f28'}, inplace = True)
cleaning7['model'].replace({'bo-105': '105'}, inplace = True)
cleaning7['model'].replace({'oh-6a': 'oh6'}, inplace = True)
cleaning7['model'].replace({'mbb-bk_117_c-2': 'bk117'}, inplace = True)
cleaning7['model'].replace({'uh-12b': 'uh12'}, inplace = True)
cleaning7['model'].replace({'109e': 'a109'}, inplace = True)
cleaning7['model'].replace({'as350_b3': 'as350'}, inplace = True)
cleaning7['model'].replace({'oh-58': 'oh58'}, inplace = True)
cleaning7['model'].replace({'r-44_ii': 'r44'}, inplace = True)
cleaning7['model'].replace({'sa315b': 'sa315'}, inplace = True)
cleaning7['model'].replace({'s-55b': 's55'}, inplace = True)
cleaning7['model'].replace({'as_355f1': 'as355'}, inplace = True)
cleaning7['model'].replace({'206-3b': '206'}, inplace = True)
cleaning7['model'].replace({'sh-3h': 'seaking'}, inplace = True)
cleaning7['model'].replace({'uh-12d': 'uh12'}, inplace = True)
cleaning7['model'].replace({'214st': '214'}, inplace = True)
cleaning7['model'].replace({'s-58jt': 's58'}, inplace = True)
cleaning7['model'].replace({'fh-1100': 'fh1100'}, inplace = True)
cleaning7['model'].replace({'s-58b': 's58'}, inplace = True)
cleaning7['model'].replace({'47g-3b-2': '47'}, inplace = True)
cleaning7['model'].replace({'th-55': '500'}, inplace = True)
cleaning7['model'].replace({'206b-iii': '206'}, inplace = True)
cleaning7['model'].replace({'r22_alpha': 'r22'}, inplace = True)
cleaning7['model'].replace({'aw119mkii': 'a119'}, inplace = True)
cleaning7['model'].replace({'s-76b': 's76'}, inplace = True)
cleaning7['model'].replace({'ah-1f': 'ah1'}, inplace = True)
cleaning7['model'].replace({'sw-4': 'sw4'}, inplace = True)
cleaning7['model'].replace({'r44 ii': 'r44'}, inplace = True)
cleaning7['model'].replace({'th55a/269a': '269'}, inplace = True)
cleaning7['model'].replace({'as_350_ba': 'as350'}, inplace = True)
cleaning7['model'].replace({'206_b3': '206'}, inplace = True)
cleaning7['model'].replace({'47g-5': '47'}, inplace = True)
cleaning7['model'].replace({'as-350-b2': 'as350'}, inplace = True)
cleaning7['model'].replace({'hughes_269c': '269'}, inplace = True)
cleaning7['model'].replace({'47g-4': '47'}, inplace = True)
cleaning7['model'].replace({'uh-34d': 'uh34'}, inplace = True)
cleaning7['model'].replace({'uh34': 'h34'}, inplace = True)
cleaning7['model'].replace({'206-b': '206'}, inplace = True)
cleaning7['model'].replace({'f28c': 'f28'}, inplace = True)
cleaning7['model'].replace({'aw109': 'a109'}, inplace = True)
cleaning7['model'].replace({'206b3': '206'}, inplace = True)
cleaning7['model'].replace({'uh12e': 'uh12'}, inplace = True)
cleaning7['model'].replace({'bo 105': 'bo105'}, inplace = True)
cleaning7['model'].replace({'412_-_ep': '412'}, inplace = True)
cleaning7['model'].replace({'s-58et': 's58'}, inplace = True)
cleaning7['model'].replace({'sa-341g': 'sa314'}, inplace = True)
cleaning7['model'].replace({'92': 's92'}, inplace = True)
cleaning7['model'].replace({'mbb-bk': 'bk117'}, inplace = True)
cleaning7['model'].replace({'300c': '300'}, inplace = True)
cleaning7['model'].replace({'222u': '222'}, inplace = True)
cleaning7['model'].replace({'tomcat_mk6b': 'tomcat'}, inplace = True)
cleaning7['model'].replace({'s64e': 's64'}, inplace = True)
cleaning7['model'].replace({'k-1200': 'k1200'}, inplace = True)
cleaning7['model'].replace({'as-350-b3': 'as350'}, inplace = True)
cleaning7['model'].replace({'r22b': 'r22'}, inplace = True)
cleaning7['model'].replace({'pv-18': 'pv18'}, inplace = True)
cleaning7['model'].replace({'369a_(oh-6a)': '369'}, inplace = True)
cleaning7['model'].replace({'bo-105s': 'bo105'}, inplace = True)
cleaning7['model'].replace({'530_(369ff)': '369'}, inplace = True)
cleaning7['model'].replace({'s-55': 's55'}, inplace = True)
cleaning7['model'].replace({'ch-54b': 'h54'}, inplace = True)
cleaning7['model'].replace({'47j-2a': '47'}, inplace = True)
cleaning7['model'].replace({'sa365n-1_d': 'sa365'}, inplace = True)
cleaning7['model'].replace({'as-350-ba': 'as350'}, inplace = True)
cleaning7['model'].replace({'47g-3b': '47'}, inplace = True)
cleaning7['model'].replace({'s-61n': 's61'}, inplace = True)
cleaning7['model'].replace({'md_902': '902'}, inplace = True)
cleaning7['model'].replace({'s-76a': 's76'}, inplace = True)
cleaning7['model'].replace({'r-22_beta': 'r22'}, inplace = True)
cleaning7['model'].replace({'s-76a': 's76'}, inplace = True)
cleaning7['model'].replace({'r-22_beta': 'r22'}, inplace = True)
cleaning7['model'].replace({'s-76a': 's76'}, inplace = True)
cleaning7['model'].replace({'r22_marine': 'r22'}, inplace = True)
cleaning7['model'].replace({'as355f1': 'as355'}, inplace = True)
cleaning7['model'].replace({'s-58ht': 's58'}, inplace = True)
cleaning7['model'].replace({'206l1': '206'}, inplace = True)
cleaning7['model'].replace({'222_ut': '222'}, inplace = True)
cleaning7['model'].replace({'r44_raven_ii': 'r44'}, inplace = True)
cleaning7['model'].replace({'300_cbi': '300'}, inplace = True)
cleaning7['model'].replace({'rh-1100': 'fh1100'}, inplace = True)
cleaning7['model'].replace({'bh206b-3': '206'}, inplace = True)
cleaning7['model'].replace({'r22_beta_2': 'r22'}, inplace = True)
cleaning7['model'].replace({'eurocopter_bo-105s': 'bo105'}, inplace = True)
cleaning7['model'].replace({'r-22b': 'r22'}, inplace = True)
cleaning7['model'].replace({'md500': '500'}, inplace = True)
cleaning7['model'].replace({'47g2a': '47'}, inplace = True)
cleaning7['model'].replace({'f28a': 'f28'}, inplace = True)
cleaning7['model'].replace({'arh-70_exp.': 'arh70'}, inplace = True)
cleaning7['model'].replace({'ec135t1': 'ec135'}, inplace = True)
cleaning7['model'].replace({'s61n': 's61'}, inplace = True)
cleaning7['model'].replace({'hx-1_wasp': 'hx1'}, inplace = True)
cleaning7['model'].replace({'ec135t2': 'ec135'}, inplace = True)
cleaning7['model'].replace({'105': 'bo105'}, inplace = True)
cleaning7['model'].replace({'as350d': 'as350'}, inplace = True)
cleaning7['model'].replace({'uh_12-e': 'uh12'}, inplace = True)
cleaning7['model'].replace({'s-76c+': 's76'}, inplace = True)
cleaning7['model'].replace({'47g-5a': '47'}, inplace = True)
cleaning7['model'].replace({'ec-130_b4 ': 'ec130'}, inplace = True)
cleaning7['model'].replace({'500d': '500'}, inplace = True)
cleaning7['model'].replace({'bo_105_ls': 'bo105'}, inplace = True)
cleaning7['model'].replace({'sa_316b': 'sa316'}, inplace = True)
cleaning7['model'].replace({'b0-105cbs-4': 'bo105'}, inplace = True)
cleaning7['model'].replace({'f-280 ': 'f28'}, inplace = True)
cleaning7['model'].replace({'bk117-a3': 'bk117'}, inplace = True)
cleaning7['model'].replace({'f-280': 'f28'}, inplace = True)
cleaning7['model'].replace({'ec-130_b4': 'ec130'}, inplace = True)
cleaning7['model'].replace({'m-74': 'oh13'}, inplace = True)
cleaning7['model'].replace({'oh-13e-m74': 'oh13'}, inplace = True)
cleaning7['model'].replace({'204b': '204'}, inplace = True)
cleaning7['model'].replace({'b407': '407'}, inplace = True)
cleaning7['model'].replace({'206l4': '206'}, inplace = True)
cleaning7['model'].replace({'206-l3': '206'}, inplace = True)
cleaning7['model'].replace({'bh-222-u': '222'}, inplace = True)
cleaning7['model'].replace({'412hp': '412'}, inplace = True)
cleaning7['model'].replace({'uh-ih': 'uh1'}, inplace = True)
cleaning7['model'].replace({'222ut': '222'}, inplace = True)
cleaning7['model'].replace({'sa316b': 'sa316'}, inplace = True)
cleaning7['model'].replace({'as-355f': 'as355'}, inplace = True)
cleaning7['model'].replace({'sa_315_b': 'sa315'}, inplace = True)
cleaning7['model'].replace({'mi-8mtv-1': 'mi8'}, inplace = True)
cleaning7['model'].replace({'th-13t': 'h13'}, inplace = True)
cleaning7['model'].replace({'s350d': 'sa350'}, inplace = True)
cleaning7['model'].replace({'47g3b1': '47'}, inplace = True)
cleaning7['model'].replace({'mk68': 'tomcat'}, inplace = True)
cleaning7['model'].replace({'uh-12-soloy': 'uh12'}, inplace = True)
cleaning7['model'].replace({'12b': 'uh12'}, inplace = True)
cleaning7['model'].replace({'fh_1100': 'fh1100'}, inplace = True)
cleaning7['model'].replace({'uh12l4': 'uh12'}, inplace = True)
cleaning7['model'].replace({'12e': 'uh12'}, inplace = True)
cleaning7['model'].replace({'sa-315b': 'sa315'}, inplace = True)
cleaning7['model'].replace({'47j-2': '47'}, inplace = True)
cleaning7['model'].replace({'f280': 'f28'}, inplace = True)
cleaning7['model'].replace({'sa_315b': 'sa315'}, inplace=True)
cleaning7['model'].replace({'oh-13e': 'oh13'}, inplace=True)
cleaning7['model'].replace({'as-350d': 'as350'}, inplace=True)
cleaning7['model'].replace({'h-19': 'h19'}, inplace=True)
cleaning7['model'].replace({'sa-318c': 'sa318'}, inplace=True)
cleaning7['model'].replace({'sa-330j': 'sa330'}, inplace=True)
cleaning7['model'].replace({'341g': 'sa341'}, inplace=True)
cleaning7['model'].replace({'315b': 'sa315'}, inplace=True)
cleaning7['model'].replace({'tomcat_mk6c': 'tomcat'}, inplace=True)
cleaning7['model'].replace({'47d-1': '47'}, inplace=True)
cleaning7['model'].replace({'(soloy_conversion)': '47'}, inplace=True)
cleaning7['model'].replace({'sa-319b': 'sa319'}, inplace=True)
cleaning7['model'].replace({'k600-3': 'k600'}, inplace=True)
cleaning7['model'].replace({'h-19a': 'h19'}, inplace=True)
cleaning7['model'].replace({'uh19': 'h19'}, inplace=True)
cleaning7['model'].replace({'222b': '222'}, inplace=True)
cleaning7['model'].replace({'b212': '212'}, inplace=True)
cleaning7['model'].replace({'uh1h': 'uh1'}, inplace=True)


In [1767]:
cleaning7.model.value_counts()

206          408
r22          358
r44          356
269          181
47           180
369          178
as350        164
amateur      127
407          77 
uh12         56 
f28          41 
uh1          38 
s76          31 
bk117        27 
212          26 
500          26 
r66          25 
ec130        25 
280          23 
b2           21 
a109         21 
ec135        18 
222          16 
600          16 
as355        14 
bo105        14 
oh13         14 
oh58         13 
sa315        13 
412          12 
430          8  
oh6          8  
300          8  
ec120        7  
fh1100       6  
429          6  
s55          6  
214          6  
139          5  
s58          5  
s61          5  
cabri        5  
sa316        5  
a119         4  
204          4  
tomcat       4  
230          4  
h34          3  
sa318        3  
h19          3  
s92          3  
seaking      3  
k1200        3  
as365        3  
205          3  
480b         3  
dhruv        2  
289fx        2  
oh23         2

In [1768]:
cleaning8=cleaning7.copy()


In [1769]:
cleaning8

,investigation_type,event_date,location,country,latitude,longitude,make,model,broad_phase_of_flight,amateur_built,number_of_engines,engine_type,far_description,purpose_of_flight,injury_severity,aircraft_damage,weather_condition,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,people_on_board,registration_number
0,accident,04/16/2019,"Hauula, HI",United States,21.588333,-157.911944,md_helicopter,369,CRUISE,no,1,Turbo Shaft,Part 135: Air Taxi & Commuter,NaN,non-fatal,Substantial,VMC,0,0,0,4,4,N593C
1,accident,04/04/2019,"Tampa, FL",United States,27.943333,-82.401944,robinson,r44,CRUISE,no,1,Reciprocating,Part 91: General Aviation,Personal,fatal(1),Substantial,VMC,1,1,0,2,4,N4046J
2,accident,04/02/2019,"Taunton, MA",United States,41.874445,-71.016389,robinson,r44,NaN,no,1,NaN,Part 91: General Aviation,NaN,non-fatal,Substantial,VMC,0,0,0,2,2,N344R
3,accident,03/30/2019,"Montgomery, NY",United States,41.539444,-74.306944,bell,222,TAKEOFF,no,2,Turbo Shaft,Part 91: General Aviation,Personal,non-fatal,Substantial,VMC,0,0,1,5,6,N222ML
4,accident,03/29/2019,"Valley Center, CA",United States,NaN,NaN,bell,206,NaN,no,NaN,NaN,Part 137: Agricultural,Aerial Application,non-fatal,Substantial,NaN,0,0,1,0,1,N83181
5,accident,03/22/2019,"Blazkov village, Czech Republic., Czech Republic",Czech Republic,NaN,NaN,enstrom,480,MANEUVERING,no,NaN,NaN,"Non-U.S., Non-Commercial",Instructional,fatal(2),Substantial,NaN,2,0,0,0,2,NaN
6,accident,03/20/2019,"Madill, OK",United States,NaN,NaN,robinson,r22,CRUISE,no,1,Reciprocating,Part 91: General Aviation,Personal,non-fatal,Substantial,VMC,0,0,0,1,1,N7782H
7,accident,03/17/2019,"Newberg, OR",United States,NaN,NaN,md_helicopter,369,CRUISE,no,1,Turbo Shaft,Part 91: General Aviation,Personal,non-fatal,Substantial,VMC,0,0,0,2,2,N119T
8,accident,03/13/2019,"Keene, NH",United States,42.898333,-72.270834,hughes,500,CRUISE,no,1,Reciprocating,Part 91: General Aviation,Personal,non-fatal,Substantial,VMC,0,0,0,1,1,N8045H
9,accident,03/13/2019,"Ponte Vedra Beach, FL",United States,30.206111,-81.403333,bell,206,MANEUVERING,no,NaN,Turbo Shaft,Part 91: General Aviation,Public Aircraft - Local,non-fatal,Substantial,VMC,0,0,0,2,2,N519P


##  

## Engine Type

In [1770]:
cleaning8.engine_type.value_counts()

 Reciprocating     1246
 Turbo Shaft       1110
 Unknown           18  
 Turbo Prop        6   
 Turbo Jet         6   
Name: engine_type, dtype: int64

In [1771]:
cleaning8.engine_type=[str(i).strip().replace(' ','_').lower() for i in cleaning8.engine_type]

In [1780]:
print(cleaning8.engine_type1.unique())
cleaning8.engine_type.value_counts()

['turbine' 'reciprocating' 'nan' 'unknown']


reciprocating    1246
turbo_shaft      1110
nan              316 
unknown          18  
turbo_jet        6   
turbo_prop       6   
Name: engine_type, dtype: int64

In [1773]:
# function to split reciprocating vs turbine
def engine(x):
    
    if 'reciprocating' in x:
        return 'reciprocating'
    elif ('turbo_shaft' in x) or ('turbo_jet' in x) or ('turbo_prop' in x):
        return 'turbine'
    elif ('unknown' in x): 
        return 'unknown'
    else:
        return 'nan'

In [1774]:
cleaning8['engine_type1'] = cleaning8.engine_type.map(engine)
print(cleaning8.engine_type1.unique())
print(cleaning8.engine_type1.value_counts())

['turbine' 'reciprocating' 'nan' 'unknown']
reciprocating    1246
turbine          1122
nan              316 
unknown          18  
Name: engine_type1, dtype: int64


In [1777]:
cleaning8.columns

Index(['investigation_type', 'event_date', 'location', 'country', 'latitude', 'longitude', 'make', 'model', 'broad_phase_of_flight', 'amateur_built', 'number_of_engines', 'engine_type', 'far_description', 'purpose_of_flight', 'injury_severity', 'aircraft_damage', 'weather_condition', 'total_fatal_injuries', 'total_serious_injuries', 'total_minor_injuries', 'total_uninjured', 'people_on_board', 'registration_number', 'engine_type1'], dtype='object')

In [1778]:
# removing engine_type as it is no longer needed, and moving columns around for better 
# visibiity of engine_type1 with make and model 
cleaning9 = cleaning8[['investigation_type', 'event_date', 'location', 'country', 'latitude',
                       'longitude', 'make', 'model', 'engine_type1','broad_phase_of_flight', 'amateur_built', 
                       'number_of_engines', 'far_description', 'purpose_of_flight', 
                       'injury_severity', 'aircraft_damage', 'weather_condition', 'total_fatal_injuries', 
                       'total_serious_injuries', 'total_minor_injuries', 'total_uninjured', 
                       'people_on_board', 'registration_number']]

In [1779]:
# checking to see where the nan values are in the engine_type column
cleaning9[cleaning9['engine_type1']=='nan']

,investigation_type,event_date,location,country,latitude,longitude,make,model,engine_type1,broad_phase_of_flight,amateur_built,number_of_engines,far_description,purpose_of_flight,injury_severity,aircraft_damage,weather_condition,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,people_on_board,registration_number
2,accident,04/02/2019,"Taunton, MA",United States,41.874445,-71.016389,robinson,r44,nan,NaN,no,1,Part 91: General Aviation,NaN,non-fatal,Substantial,VMC,0,0,0,2,2,N344R
4,accident,03/29/2019,"Valley Center, CA",United States,NaN,NaN,bell,206,nan,NaN,no,NaN,Part 137: Agricultural,Aerial Application,non-fatal,Substantial,NaN,0,0,1,0,1,N83181
5,accident,03/22/2019,"Blazkov village, Czech Republic., Czech Republic",Czech Republic,NaN,NaN,enstrom,480,nan,MANEUVERING,no,NaN,"Non-U.S., Non-Commercial",Instructional,fatal(2),Substantial,NaN,2,0,0,0,2,NaN
10,accident,03/10/2019,"Galliano, LA",United States,29.422778,-90.178334,bell,407,nan,NaN,no,1,Part 135: Air Taxi & Commuter,NaN,fatal(2),Destroyed,NaN,2,0,0,0,2,N577AL
13,accident,03/04/2019,"Timmins, CYTS, Canada",Canada,NaN,NaN,robinson,r66,nan,CRUISE,no,NaN,"Non-U.S., Non-Commercial",NaN,fatal(2),Substantial,NaN,2,0,0,0,2,NaN
16,accident,02/21/2019,"Kukuihaele, HI",United States,20.110833,-155.554167,md_helicopter,369,nan,APPROACH,no,NaN,Part 91: General Aviation,Positioning,non-fatal,Substantial,NaN,0,1,0,0,1,N506PH
18,accident,02/18/2019,"Glennallen, AK",United States,NaN,NaN,eurocopter,as350,nan,NaN,no,1,Part 135: Air Taxi & Commuter,NaN,unavailable,NaN,NaN,0,0,0,0,0,N350ST
19,accident,02/17/2019,"Roeldal, Norway",Norway,59.869723,6.721389,robinson,r44,nan,CRUISE,no,NaN,"Non-U.S., Non-Commercial",NaN,fatal(2),Destroyed,NaN,2,0,0,0,2,NaN
20,accident,02/11/2019,"Barueri County, Brazil",Brazil,NaN,NaN,bell,206,nan,LANDING,no,NaN,"Non-U.S., Non-Commercial",NaN,fatal(3),Destroyed,NaN,3,0,0,0,3,NaN
29,accident,01/02/2019,"Santa Maria Island, FL",United States,27.483334,-82.733334,robinson,r44,nan,NaN,no,1,Part 91: General Aviation,NaN,unavailable,Destroyed,NaN,0,0,0,0,0,N772MG


In [1787]:
# filling some the reciprocating engine types
cleaning9.loc[cleaning9['model'].str.contains('r22'), 'engine_type1']= 'reciprocating'
cleaning9.loc[cleaning9['model'].str.contains('269'), 'engine_type1']= 'reciprocating'
cleaning9.loc[cleaning9['model'].str.contains('amateur'), 'engine_type1']= 'reciprocating'
cleaning9.loc[cleaning9['model'].str.contains('b2'), 'engine_type1']= 'reciprocating'
cleaning9.loc[cleaning9['model'].str.contains('cabri'), 'engine_type1']= 'reciprocating'
cleaning9.loc[cleaning9['model'].str.contains('uh12'), 'engine_type1']= 'reciprocating'
cleaning9.loc[cleaning9['model'].str.contains('r44'), 'engine_type1']= 'reciprocating'

In [1797]:
# filling remaining nan vlaues with 'turbine'
cleaning9['engine_type1'] = cleaning9['engine_type1'].replace('nan', np.nan).fillna('turbine')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [1798]:
# checking the nan values have gone
cleaning9[cleaning9['engine_type1']=='nan']

,investigation_type,event_date,location,country,latitude,longitude,make,model,engine_type1,broad_phase_of_flight,amateur_built,number_of_engines,far_description,purpose_of_flight,injury_severity,aircraft_damage,weather_condition,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,people_on_board,registration_number


In [1800]:
# doing a random check of a turbine model. Have found incorrect information in here that needs to be fixed
cleaning9[cleaning9['model']=='r66']

,investigation_type,event_date,location,country,latitude,longitude,make,model,engine_type1,broad_phase_of_flight,amateur_built,number_of_engines,far_description,purpose_of_flight,injury_severity,aircraft_damage,weather_condition,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,people_on_board,registration_number
13,accident,03/04/2019,"Timmins, CYTS, Canada",Canada,NaN,NaN,robinson,r66,turbine,CRUISE,no,NaN,"Non-U.S., Non-Commercial",NaN,fatal(2),Substantial,NaN,2,0,0,0,2,NaN
124,accident,06/19/2018,"Cordova, AK",United States,60.675000,-144.741944,robinson,r66,turbine,LANDING,no,1,Part 135: Air Taxi & Commuter,NaN,non-fatal,Substantial,VMC,0,0,0,3,3,N7040P
265,accident,08/21/2017,"Maly Utrish, Russia",Russia,NaN,NaN,robinson,r66,turbine,CRUISE,no,NaN,"Non-U.S., Non-Commercial",NaN,fatal(1),Substantial,NaN,1,0,0,0,1,NaN
330,accident,05/20/2017,"Canon City, CO",United States,38.290000,-105.190000,robinson,r66,turbine,APPROACH,no,1,Part 91: General Aviation,Business,non-fatal,Substantial,VMC,0,0,0,5,5,N778TL
370,accident,02/12/2017,"Lake Teletskoye, Russia",Russia,51.680556,87.666389,robinson,r66,turbine,UNKNOWN,no,NaN,"Non-U.S., Non-Commercial",NaN,fatal(5),Substantial,UNK,5,0,0,0,5,NaN
398,accident,11/14/2016,"Winton, New Zealand",New Zealand,NaN,NaN,robinson,r66,turbine,MANEUVERING,no,1,NaN,NaN,non-fatal,Substantial,NaN,0,0,0,1,1,NaN
421,accident,09/08/2016,"Carinthia, Austria",Austria,47.023334,12.748333,robinson,r66,turbine,UNKNOWN,no,NaN,NaN,NaN,fatal(1),Destroyed,VMC,1,1,0,0,2,NaN
439,accident,08/07/2016,"Voronezhskaya oblast, Russia",Russia,50.874445,39.883056,robinson,r66,turbine,DESCENT,no,1,"Non-U.S., Non-Commercial",NaN,non-fatal,Substantial,UNK,0,5,0,0,5,NaN
463,accident,06/23/2016,"Wikieup, AZ",United States,34.461111,-113.683334,robinson,r66,turbine,NaN,no,1,Part 91: General Aviation,Positioning,fatal(2),Destroyed,VMC,2,0,0,0,2,N117TW
475,accident,06/07/2016,"Beaumont, TX",United States,30.001944,-94.177223,robinson,r66,reciprocating,STANDING,no,1,Part 91: General Aviation,Personal,non-fatal,Substantial,VMC,0,0,0,1,1,N38SL


In [1801]:
cleaning9.loc[cleaning9['model'].str.contains('r66'), 'engine_type1']= 'turbine'

In [1802]:
# checking they've changed to turbine accurately
cleaning9[cleaning9['model']=='r66']

,investigation_type,event_date,location,country,latitude,longitude,make,model,engine_type1,broad_phase_of_flight,amateur_built,number_of_engines,far_description,purpose_of_flight,injury_severity,aircraft_damage,weather_condition,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,people_on_board,registration_number
13,accident,03/04/2019,"Timmins, CYTS, Canada",Canada,NaN,NaN,robinson,r66,turbine,CRUISE,no,NaN,"Non-U.S., Non-Commercial",NaN,fatal(2),Substantial,NaN,2,0,0,0,2,NaN
124,accident,06/19/2018,"Cordova, AK",United States,60.675000,-144.741944,robinson,r66,turbine,LANDING,no,1,Part 135: Air Taxi & Commuter,NaN,non-fatal,Substantial,VMC,0,0,0,3,3,N7040P
265,accident,08/21/2017,"Maly Utrish, Russia",Russia,NaN,NaN,robinson,r66,turbine,CRUISE,no,NaN,"Non-U.S., Non-Commercial",NaN,fatal(1),Substantial,NaN,1,0,0,0,1,NaN
330,accident,05/20/2017,"Canon City, CO",United States,38.290000,-105.190000,robinson,r66,turbine,APPROACH,no,1,Part 91: General Aviation,Business,non-fatal,Substantial,VMC,0,0,0,5,5,N778TL
370,accident,02/12/2017,"Lake Teletskoye, Russia",Russia,51.680556,87.666389,robinson,r66,turbine,UNKNOWN,no,NaN,"Non-U.S., Non-Commercial",NaN,fatal(5),Substantial,UNK,5,0,0,0,5,NaN
398,accident,11/14/2016,"Winton, New Zealand",New Zealand,NaN,NaN,robinson,r66,turbine,MANEUVERING,no,1,NaN,NaN,non-fatal,Substantial,NaN,0,0,0,1,1,NaN
421,accident,09/08/2016,"Carinthia, Austria",Austria,47.023334,12.748333,robinson,r66,turbine,UNKNOWN,no,NaN,NaN,NaN,fatal(1),Destroyed,VMC,1,1,0,0,2,NaN
439,accident,08/07/2016,"Voronezhskaya oblast, Russia",Russia,50.874445,39.883056,robinson,r66,turbine,DESCENT,no,1,"Non-U.S., Non-Commercial",NaN,non-fatal,Substantial,UNK,0,5,0,0,5,NaN
463,accident,06/23/2016,"Wikieup, AZ",United States,34.461111,-113.683334,robinson,r66,turbine,NaN,no,1,Part 91: General Aviation,Positioning,fatal(2),Destroyed,VMC,2,0,0,0,2,N117TW
475,accident,06/07/2016,"Beaumont, TX",United States,30.001944,-94.177223,robinson,r66,turbine,STANDING,no,1,Part 91: General Aviation,Personal,non-fatal,Substantial,VMC,0,0,0,1,1,N38SL


In [1803]:
# now to see where there are other null values that need to be fixed
cleaning9.isnull().sum()

investigation_type        0  
event_date                0  
location                  6  
country                   13 
latitude                  584
longitude                 585
make                      0  
model                     0  
engine_type1              0  
broad_phase_of_flight     628
amateur_built             0  
number_of_engines         320
far_description           84 
purpose_of_flight         405
injury_severity           0  
aircraft_damage           34 
weather_condition         326
total_fatal_injuries      0  
total_serious_injuries    0  
total_minor_injuries      0  
total_uninjured           0  
people_on_board           0  
registration_number       418
dtype: int64

##  

## Weather Condition

In [1804]:
cleaning9.weather_condition.value_counts()

 VMC     2243
 IMC     96  
 UNK     37  
Name: weather_condition, dtype: int64

In [1805]:
cleaning9.weather_condition=[str(i).strip().lower() for i in cleaning9.weather_condition]

/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:5086: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [1806]:
cleaning9.weather_condition.value_counts()

vmc    2243
nan    326 
imc    96  
unk    37  
Name: weather_condition, dtype: int64

In [1807]:
# will leave the unknowns and nans in for now as not to reduce the rows in the dataframe.
# will check if they add too much noice when it gets to modelling.
cleaning9['weather_condition'] = cleaning9['weather_condition'].replace('nan', np.nan).fillna('unknown')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [1810]:
cleaning9.weather_condition.replace({'unk':'unknown'}, inplace = True)

/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:6576: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [1811]:
cleaning9.weather_condition.value_counts()

vmc        2243
unknown    363 
imc        96  
Name: weather_condition, dtype: int64

##  

## Purpose of Flight

In [1812]:
cleaning9.purpose_of_flight.value_counts()

 Personal                      585
 Instructional                 446
 Aerial Application            248
 Unknown                       218
 Positioning                   155
 Other Work Use                125
 Business                      125
 Aerial Observation            113
 External Load                 76 
 Public Aircraft - Local       46 
 Flight Test                   30 
 Public Aircraft - State       28 
 Public Aircraft - Federal     25 
 Executive/Corporate           20 
 Public Aircraft               18 
 Ferry                         17 
 Firefighting                  12 
 Air Race/Show                 5  
 Air Drop                      5  
Name: purpose_of_flight, dtype: int64

In [1813]:
cleaning9.purpose_of_flight=[str(i).strip().lower() for i in cleaning9.purpose_of_flight]

/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:5086: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [1814]:
cleaning9.purpose_of_flight.value_counts()

personal                     585
instructional                446
nan                          405
aerial application           248
unknown                      218
positioning                  155
business                     125
other work use               125
aerial observation           113
external load                76 
public aircraft - local      46 
flight test                  30 
public aircraft - state      28 
public aircraft - federal    25 
executive/corporate          20 
public aircraft              18 
ferry                        17 
firefighting                 12 
air race/show                5  
air drop                     5  
Name: purpose_of_flight, dtype: int64

In [1815]:
cleaning9['purpose_of_flight'] = cleaning9['purpose_of_flight'].replace('nan', np.nan).fillna('unknown')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [1816]:
# with a view to looking at the type (therefore experience) of pilot who is flying


In [1818]:
def private_professional(x):
    if 'instructional' in x:
        return 'instructional'
    elif 'unknown'in x:
        return 'unknown'
    elif 'personal' in x:
        return 'private'
    else:
        return 'professional'
    
cleaning9['pilot_type']= cleaning9.purpose_of_flight.map(private_professional)
print(cleaning9.pilot_type.unique())
print(cleaning9.pilot_type.value_counts())    

['unknown' 'private' 'professional' 'instructional']
professional     1048
unknown          623 
private          585 
instructional    446 
Name: pilot_type, dtype: int64


## Location, Country, Longitude and Latitude

In [1533]:

cleaning8=cleaning8[cleaning8.location != "MUSTANG BLK A11"]
cleaning8=cleaning8[cleaning8.location != "E BRAKE BLK 397	"]
cleaning8=cleaning8[cleaning8.location != "BUCCANEER OIL F"]
cleaning8=cleaning8[cleaning8.location != "E BRAKE BLK 397"]

In [1141]:
cleaning8.loc[:,'location'].apply(lambda x: x.split(',')[0] if type(x)==str else x)

0        Hauula                
1        Tampa                 
2        Taunton               
3        Montgomery            
4        Valley Center         
5        Blazkov village       
6        Madill                
7        Newberg               
8        Keene                 
9        Ponte Vedra Beach     
10       Galliano              
11       Forks                 
12       Talking Rock          
13       Timmins               
14       Union Center          
15       TULLAHOMA             
16       Kukuihaele            
17       Clearwater            
18       Glennallen            
19       Roeldal               
20       Barueri County        
21       Tampa                 
22       Zaleski               
23       Ashland               
24       Yuma                  
25       Ponca City            
26       Belen                 
27       Uvalde                
28       Grace                 
29       Santa Maria Island    
30       Ubatuba               
31      

In [1142]:
cleaning8.location = [i.strip() if type(i) != float else i for i in cleaning8.location]

In [1143]:
cleaning8['location'].replace({'Timmins, CYTS, Canada': 'timmins'}, inplace=True)
cleaning8['location'].replace({'Barueri County, Brazil': 'Barueri County'}, inplace=True)
cleaning8['location'].replace({'Alice Springs, Australia': 'alice springs'}, inplace=True)
cleaning8['location'].replace({'Leicester, United Kingdom': 'leicester'}, inplace=True)
cleaning8['location'].replace({'Bodaybo, Russia': 'bodaybo'}, inplace=True)
cleaning8['location'].replace({'Buttonville, Canada': 'Buttonville'}, inplace=True)
cleaning8['location'].replace({'Kyiv, Ukraine': 'Kyiv'}, inplace=True)
cleaning8['location'].replace({'Domažlická 1059, Plzen - Skvrnany, Czech ': 'Domažlická 1059, Plzen - Skvrnany'}, inplace=True)
cleaning8['location'].replace({'Ulladulla, Australia': 'Ulladulla'}, inplace=True)
cleaning8['location'].replace({'Queenstown, South Africa': 'Queenstown'}, inplace=True)
cleaning8['location'].replace({'Naunton Beauchamp, Worcestershire, United Kingdom': 'Naunton Beauchamp, Worcestershire'}, inplace=True)
cleaning8['location'].replace({'Wolfsberg, Austria': 'Wolfsberg'}, inplace=True)
cleaning8['location'].replace({'Waiouru, New Zealand': 'Waiouru'}, inplace=True)
cleaning8['location'].replace({'Ciudad Valles, Mexico': 'Ciudad Valles'}, inplace=True)
cleaning8['location'].replace({'Valle Nuevo, Dominican Republic ': 'Valle Nuevo'}, inplace=True)
cleaning8['location'].replace({'Mabula, South Africa': 'Mabula'}, inplace=True)
cleaning8['location'].replace({'Vereda Plateado, Colombia': 'Vereda Plateado'}, inplace=True)
cleaning8['location'].replace({'Hardy Reef, Queensland, Australia': 'Hardy Reef, Queensland'}, inplace=True)
cleaning8['location'].replace({'Uetersen, Germany': 'Uetersen'}, inplace=True)
cleaning8['location'].replace({'Marchwood, New Hampshire, United Kingdom': 'Marchwood, New Hampshire'}, inplace=True)
cleaning8['location'].replace({'Shetland, United Kingdom': 'Shetland'}, inplace=True)
cleaning8['location'].replace({'Recife, Brazil': 'Recife'}, inplace=True)
cleaning8['location'].replace({'Oberhausen-Rheinhausen, Germany': 'Oberhausen-Rheinhausen'}, inplace=True)
cleaning8['location'].replace({'Alicante, Spain': 'Alicante'}, inplace=True)
cleaning8['location'].replace({'Bacoor, Philippines': 'Bacoor'}, inplace=True)
cleaning8['location'].replace({'Krasny Pakhar village, Russia': 'Krasny Pakhar village'}, inplace=True)
cleaning8['location'].replace({'Herzelia, Israel': 'Herzelia'}, inplace=True)
cleaning8['location'].replace({'Maly Utrish, Russia': 'Maly Utrish'}, inplace=True)
cleaning8['location'].replace({'Bonvilliers, France': 'Bonvilliers'}, inplace=True)
cleaning8['location'].replace({'Dongducheon, Korea, Republic Of': 'Dongducheon'}, inplace=True)
cleaning8['country'].replace({'Korea, Republic Of': 'Korea'}, inplace=True)
cleaning8['location'].replace({'Fengbin, Taiwan': 'Fengbin'}, inplace=True)
cleaning8['location'].replace({'Nilanga, India': 'Nilanga'}, inplace=True)
cleaning8['location'].replace({'Kwa-Zulu Natal, South Africa': 'Kwa-Zulu Natal'}, inplace=True)
cleaning8['location'].replace({'Heilbron, South Africa': 'Heilbron'}, inplace=True)
cleaning8['location'].replace({'Presov, Slovakia': 'Presov'}, inplace=True)
cleaning8['location'].replace({'Beloretsk, Russia': 'Beloretsk'}, inplace=True)
cleaning8['location'].replace({'Reefton, New Zealand': 'Reefton'}, inplace=True)
cleaning8['location'].replace({'Figeac, France': 'Figeac'}, inplace=True)
cleaning8['location'].replace({'Blacksod Bay, Ireland': 'Blacksod Bay'}, inplace=True)
cleaning8['location'].replace({'Raetihi, New Zealand': 'Raetihi'}, inplace=True)
cleaning8['location'].replace({'Locarno, Switzerland': 'Locarno'}, inplace=True)
cleaning8['location'].replace({'Vinogradny Township, Russia': 'Vinogradny Township'}, inplace=True)
cleaning8['location'].replace({'Winton, New Zealand': 'Winton'}, inplace=True)
cleaning8['location'].replace({'Kamo, New Zealand, New Zealand': 'Kamo'}, inplace=True)
cleaning8['location'].replace({'Wanaka, New Zealand': 'Wanaka'}, inplace=True)
cleaning8['location'].replace({'Ballina, Australia': 'Ballina'}, inplace=True)
cleaning8['location'].replace({'Yartsevo, Russia': 'Yartsevo'}, inplace=True)
cleaning8['location'].replace({'Lausanne - La Blecherette, Switzerland': 'Lausanne - La Blecherette'}, inplace=True)
cleaning8['location'].replace({'Sayaxché, Guatemala': 'Sayaxché'}, inplace=True)
cleaning8['location'].replace({'Senheim, Germany': 'Senheim'}, inplace=True)
cleaning8['location'].replace({'Manchester, United Kingdom': 'Manchester'}, inplace=True)
cleaning8['location'].replace({'Taldykorgan, Kazakhstan': 'Taldykorgan'}, inplace=True)
cleaning8['location'].replace({'Salinas Victoria, Mexico': 'Salinas Victoria'}, inplace=True)
cleaning8['location'].replace({'Annaka City, Japan': 'Annaka City'}, inplace=True)
cleaning8['location'].replace({'Fox Glacier, New Zealand': 'Fox Glacier'}, inplace=True)
cleaning8['location'].replace({'Tronzano Vercellese, Italy': 'Tronzano Vercellese'}, inplace=True)
cleaning8['location'].replace({'Asquinth, Canada': 'Asquinth'}, inplace=True)
cleaning8['location'].replace({'San Agustine Guatemala, Guatemala': 'San Agustine'}, inplace=True)
cleaning8['location'].replace({'Sept-Iles, Canada': 'Sept-Iles'}, inplace=True)
cleaning8['location'].replace({'Natolin village, Poland': 'Natolin village'}, inplace=True)
cleaning8['location'].replace({'Sakha Republic, Russia': 'Sakha Republic'}, inplace=True)
cleaning8['location'].replace({'Istra, Russia': 'Istra'}, inplace=True)
cleaning8['location'].replace({'Lake Peten Izta, Guatemala': 'Peten'}, inplace=True)
cleaning8['location'].replace({'Ramatuelle, France, France': 'Ramatuelle'}, inplace=True)
cleaning8['location'].replace({'Leixlip, Ireland': 'Leixlip'}, inplace=True)
cleaning8['location'].replace({'Tifalmin Village Papua New Guinea, Papua New Guinea': 'telefomin'}, inplace=True)
cleaning8['location'].replace({'Zhejiang Province, P.R. China, China': 'Zhejiang Province'}, inplace=True)
cleaning8['location'].replace({'Hradec Kralove, Czech Republic, Czech Republic': 'Hradec'}, inplace=True)
cleaning8['location'].replace({'Makarakskiy, Russia': 'makarovskiy'}, inplace=True)
cleaning8['location'].replace({'Morobe Province, Papua New Guinea': 'Morobe Province'}, inplace=True)
cleaning8['location'].replace({'Wabo, Papua New Guinea, Papua New Guinea': 'ihu'}, inplace=True)
cleaning8['location'].replace({'Bohusovice nad Ohri, Czech Republic': 'Bohusovice nad Ohri'}, inplace=True)
cleaning8['location'].replace({'Bern, Switzerland': 'Bern'}, inplace=True)
cleaning8['location'].replace({'Tena, Ecuador': 'Tena'}, inplace=True)
cleaning8['location'].replace({'Waikaia, New Zealand': 'Waikaia'}, inplace=True)
cleaning8['location'].replace({'Santa Catarina Ixtahuacan, Guatemala': 'Santa Catarina Ixtahuacan'}, inplace=True)
cleaning8['location'].replace({'Boca Chica, Dominican Republic': 'Boca Chica'}, inplace=True)
cleaning8['location'].replace({'Amaga, Colombia': 'Amaga'}, inplace=True)
cleaning8['location'].replace({'Flamborough, United Kingdom': 'Flamborough'}, inplace=True)
cleaning8['location'].replace({'Umea, Sweden': 'Umea'}, inplace=True)
cleaning8['location'].replace({'Roeselare, Belgium': 'Roeselare'}, inplace=True)
cleaning8['location'].replace({'Camp Fenelon, Canada': 'Camp Fenelon'}, inplace=True)
cleaning8['location'].replace({'Ringaskiddy, Ireland, Ireland': 'Ringaskiddy'}, inplace=True)
cleaning8['location'].replace({'vashka Township, Russia': 'vashka Township'}, inplace=True)
cleaning8['location'].replace({'Cerfontaine, Belgium, Belgium': 'Cerfontaine'}, inplace=True)
cleaning8['location'].replace({'Beijing, P.R. China, China': 'Beijing'}, inplace=True)
cleaning8['location'].replace({'Borroloola, Australia, Australia': 'Borroloola'}, inplace=True)
cleaning8['location'].replace({'Gisborne, New Zealand, New Zealand': 'Gisborne'}, inplace=True)
cleaning8['location'].replace({'Gillingham, Norfolk, United Kingdom': 'Gillingham, Norfolk'}, inplace=True)
cleaning8['location'].replace({'Berastagi, Indonesia': 'Berastagi'}, inplace=True)
cleaning8['location'].replace({'Glasgow, United Kingdom': 'Glasgow'}, inplace=True)
cleaning8['location'].replace({'Neuhausen ob Eck, Germany': 'Neuhausen ob Eck'}, inplace=True)
cleaning8['location'].replace({'Rio Ablajo, Honduras': 'Rio Ablajo'}, inplace=True)
cleaning8['location'].replace({'Seoul, Korea, Republic Of': 'Seoul'}, inplace=True)
cleaning8['country'].replace({'Korea, Republic Of': 'Korea'}, inplace=True)
cleaning8['location'].replace({'Tatacka, Taiwan': 'Tataka'}, inplace=True)
cleaning8['location'].replace({'Stavanger, Norway': 'Stavanger'}, inplace=True)
cleaning8['location'].replace({'Murbad, Thane District, India, India': 'Murbad, Thane District'}, inplace=True)
cleaning8['location'].replace({'McClure Strait, Canada': 'McClure Strait'}, inplace=True)
cleaning8['location'].replace({'Shetland Islands, United Kingdom': 'Shetland Islands'}, inplace=True)
cleaning8['location'].replace({'Kedaranath, India, India': 'kedarnath'}, inplace=True)
cleaning8['location'].replace({'Moosonee, Canada': 'Moosonee'}, inplace=True)
cleaning8['location'].replace({'Keongbuk, Korea, Republic Of': 'gyeongbuk'}, inplace=True)
cleaning8['country'].replace({'Korea, Republic Of': 'Korea'}, inplace=True)
cleaning8['location'].replace({'Bulli Tops, Australia, Australia': 'Bulli Tops'}, inplace=True)
cleaning8['location'].replace({'London, United Kingdom': 'London'}, inplace=True)
cleaning8['location'].replace({'Katra, India, India': 'Katra'}, inplace=True)
cleaning8['location'].replace({'Mc Cool Junction, NE': 'McCool Junction, Nebraska'}, inplace=True)
cleaning8['location'].replace({'2NM SW OF LYNCH, KY': 'LYNCH, KY'}, inplace=True)
cleaning8['location'].replace({'NEAR WINTER PAR, CO': 'Winter Park, Co'}, inplace=True)
cleaning8['location'].replace({'NEAR BORNITE, AK': 'BORNITE, AK'}, inplace=True)
cleaning8['location'].replace({'35 W. OF KETCHI, AK': 'KECHI, AK'}, inplace=True)
cleaning8['location'].replace({'10 SW OF MILAN, MO': 'MILAN, MO'}, inplace=True)
cleaning8['location'].replace({'GULF OF MEXICO': 'Mexico, MEXICO'}, inplace=True)
cleaning8['location'].replace({'STEAMBOAT SPRIN, CO': 'STEAMBOAT SPRINGS, CO'}, inplace=True)
cleaning8['location'].replace({'NEAR MERTZON, TX': 'MERTZON, TX'}, inplace=True)
cleaning8['location'].replace({'2NM NNM REDMOND, WA': 'REDMOND, WA'}, inplace=True)
cleaning8['location'].replace({'WEST CAMERON #4': 'WEST CAMERON, united states'}, inplace=True)
cleaning8['location'].replace({'MARMATH, ND': 'MARMARTH, ND'}, inplace=True)
cleaning8['location'].replace({'OSAGE BEECH, MO': 'OSAGE BEACH, MO'}, inplace=True)
cleaning8['location'].replace({'6NM SSE WRANGEL, AK': 'WRANGELL, AK'}, inplace=True)
cleaning8['location'].replace({'VERMAL, UT': 'VERNAL, UT'}, inplace=True)
cleaning8['location'].replace({'HUNTINGTON BEAC, CA': 'HUNTINGTON BEACH, CA'}, inplace=True)
cleaning8['location'].replace({'CHRISTMAS VALLE, OR': 'CHRISTMAS VALLEY, OR'}, inplace=True)
cleaning8['location'].replace({'INTRACOASTAL CI, LA': 'INTRACOASTAL CITY, LA'}, inplace=True)
cleaning8['location'].replace({'NEAR SAVOONGA, AK': 'SAVOONGA, AK'}, inplace=True)
cleaning8['location'].replace({'WEST DELTA 105D': 'WEST DELTA'}, inplace=True)
cleaning8['location'].replace({'NEAR KISSIMMEE, FL': 'KISSIMMEE, FL'}, inplace=True)
cleaning8['location'].replace({'NEAR TANANA, AK': 'TANANA, AK'}, inplace=True)
cleaning8['location'].replace({'NEAR TOUTLE, WA': 'TOUTLE, WA'}, inplace=True)
cleaning8['location'].replace({'NEAR BRIDEPORT, CA': 'BRIDGEPORT, CA'}, inplace=True)
cleaning8['location'].replace({'HIGH ISLAND BLK': 'HIGH ISLAND BLK, United States'}, inplace=True)
cleaning8['location'].replace({'GULF OF MEXICO': 'Mexico'}, inplace=True)
cleaning8['location'].replace({'10MI. NW WALSEN, CO': 'WALSEN, CO'}, inplace=True)
cleaning8['location'].replace({'NEAR KINGSTON, ID': 'KINGSTON, ID'}, inplace=True)
cleaning8['location'].replace({'S.E. OF MALTA, ID': 'MALTA, ID'}, inplace=True)
cleaning8['location'].replace({'REDHILL AERODRO, United Kingdom': 'REDHILL AERODROME'}, inplace=True)
cleaning8['location'].replace({'NORTH SEA, Netherlands': 'den helder'}, inplace=True)
cleaning8['location'].replace({'WENZHO, China': 'WENZHOU'}, inplace=True)
cleaning8['location'].replace({'11MI. SO.OFTELL, CO': 'TELL city, CO'}, inplace=True)
cleaning8['location'].replace({'NEAR RANDLE, WA': 'RANDLE, WA'}, inplace=True)
cleaning8['location'].replace({'INTERCOASTAL CI, LA': 'INTRACOASTAL CITY, LA'}, inplace=True)
cleaning8['location'].replace({'NEAR KALISPELL, MT': 'KALISPELL, MT'}, inplace=True)
cleaning8['location'].replace({'5NM EAST OF SAL, CA': 'SAL, CA'}, inplace=True)
cleaning8['location'].replace({'NEAR KEMMERER, WY': 'KEMMERER, WY'}, inplace=True)
cleaning8['location'].replace({'NEAR BROOTEN, MN': 'BROOTEN, MN'}, inplace=True)
cleaning8['location'].replace({'WEST CAMERON, united states': 'WEST CAMERON'}, inplace=True)

In [1144]:
cleaning8.query('longitude != longitude')[150:200]

,investigation_type,event_date,location,country,latitude,longitude,make,model,broad_phase_of_flight,amateur_built,number_of_engines,engine_type,far_description,purpose_of_flight,injury_severity,aircraft_damage,weather_condition,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,people_on_board,registration_number,engine_type1
1230,accident,04/06/2012,"Leningradsky, Russia",Russia,NaN,NaN,agusta,a119,NaN,no,1,Turbo Shaft,"Non-U.S., Commercial",NaN,fatal(4),Substantial,IMC,4,0,0,0,4,NaN,turbine
1231,accident,04/06/2012,"Huy, Belgium",Belgium,NaN,NaN,robinson,r22,NaN,no,NaN,Reciprocating,"Non-U.S., Commercial",NaN,fatal(1),Destroyed,VMC,1,0,0,0,1,NaN,reciprocating
1246,accident,03/02/2012,"Makakilo City, HI",United States,NaN,NaN,aerospatiale,as355,MANEUVERING,no,2,Turbo Shaft,Part 133: Rotorcraft Ext. Load,External Load,non-fatal,Substantial,VMC,0,0,0,1,1,N101MZ,turbine
1255,accident,02/18/2012,"Lahr, Germany",Germany,NaN,NaN,amateur,amateur,NaN,yes,1,NaN,"Non-U.S., Non-Commercial",Personal,fatal(1),Destroyed,VMC,1,0,0,0,1,NaN,turbine
1256,accident,02/17/2012,"Cultus Lake, Canada",Canada,NaN,NaN,eurocopter,as350,NaN,no,NaN,NaN,"Non-U.S., Non-Commercial",Unknown,fatal(1),Substantial,NaN,1,0,0,0,1,NaN,turbine
1260,accident,01/22/2012,"Irving, TX",United States,NaN,NaN,bell,206,LANDING,no,1,Turbo Shaft,Part 135: Air Taxi & Commuter,NaN,non-fatal,Substantial,VMC,0,0,0,4,4,N555NB,turbine
1268,accident,01/06/2012,"Lincolnshire, United Kingdom",United Kingdom,NaN,NaN,robinson,r22,NaN,no,1,NaN,Unknown,NaN,non-fatal,Substantial,NaN,0,0,0,1,1,NaN,reciprocating
1277,accident,12/07/2011,"Las Vegas, NV",United States,NaN,NaN,eurocopter,as350,NaN,no,1,Turbo Shaft,Part 135: Air Taxi & Commuter,NaN,fatal(5),Destroyed,VMC,5,0,0,0,5,N37SH,turbine
1279,accident,11/25/2011,"Quito, Ecuador",Ecuador,NaN,NaN,bell,212,NaN,no,2,Turbo Prop,"Non-U.S., Non-Commercial",Flight Test,non-fatal,Substantial,VMC,0,4,0,0,4,NaN,turbine
1284,accident,11/12/2011,"Andratx, Spain",Spain,NaN,NaN,hughes,500,NaN,no,NaN,Turbo Shaft,"Non-U.S., Non-Commercial",Personal,fatal(2),Substantial,NaN,2,0,0,0,2,NaN,turbine


In [1145]:
from  geopy.geocoders import Nominatim

locs = cleaning8[['location','country']][((cleaning8.longitude.isna())|
                              (cleaning8.latitude.isna()))&(cleaning8.location.isna()==False)&
                                         (cleaning8.country.isna() == False)]
loclist = [i.split(',')[0] for i in locs.iloc[:,0]]
countrylist = [i.split(',')[0] for i in locs.iloc[:,1]]

In [1146]:
from geopy.exc import GeocoderTimedOut
from tqdm import tqdm_notebook

def do_geocode(address):
    geolocator = Nominatim()
    try:
        return geolocator.geocode(address)
    except GeocoderTimedOut:
        return do_geocode(address)

In [1147]:
geolocator = Nominatim()
coords = []
for city,country in tqdm_notebook(zip(loclist,countrylist)):
    loc = do_geocode(city+','+country)
    try:
        coords.append([loc.latitude,loc.longitude])
    except:
        coords.append([np.nan,np.nan])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.19.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """Entry point for launching an IPython kernel.


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Using Nominatim with the default "geopy/1.19.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """


KeyboardInterrupt: 

In [ ]:
for index,loc in zip(locs.index,coords):
    cleaning8.loc[index,'latitude'] = loc[0]
    cleaning8.loc[index,'longitude'] = loc[1]

In [ ]:
locs2 = cleaning8[['location','country']][((cleaning8.longitude.isna())|
                              (cleaning8.latitude.isna()))&(cleaning8.location.isna()==False)&
                                         (cleaning8.country.isna() == False)]
loclist2 = [i for i in locs2.iloc[:,0]]
countrylist2 = [i.split(',')[0] for i in locs2.iloc[:,1]]

In [ ]:
geolocator = Nominatim()
coords2 = []
for city,country in tqdm_notebook(zip(loclist2,countrylist2)):
    loc = do_geocode(city+','+country)
    try:
        coords2.append([loc.latitude,loc.longitude])
    except:
        coords2.append([np.nan,np.nan])

In [ ]:
for index,loc in zip(locs2.index,coords2):
    cleaning8.loc[index,'latitude'] = loc[0]
    cleaning8.loc[index,'longitude'] = loc[1]

In [ ]:
cleaning8.isna().sum()

In [ ]:
cleaning8.shape

In [ ]:
geolocator = Nominatim()
city ="McCool Junction, Nebraska"
country ="united states"
loc = geolocator.geocode(city+','+ country)
print("latitude is :" ,loc.latitude,"\nlongtitude is:" ,loc.longitude)

In [ ]:
loclist2

In [ ]:
geolocator = Nominatim()
coords2 = []
for city,country in tqdm_notebook(zip(loclist2,countrylist2)):
    loc = do_geocode(city+','+country)
    try:
        coords2.append([loc.latitude,loc.longitude])
    except:
        coords2.append([np.nan,np.nan])

In [ ]:
cleaning8['latitude']

In [ ]:
# def long_lat(location):
geolocator = Nominatim()
city ="HITCHCOCK, TX	"
country ="united states"
loc = geolocator.geocode(city+','+ country)
print("latitude is :" ,loc.latitude,"\nlongtitude is:" ,loc.longitude)

In [ ]:
df1 = cleaning8[cleaning8.isna().any(axis=1)]
df1

In [ ]:
cleaning8['location'].replace({'2NM SW OF LYNCH, KY': 'LYNCH, KY'}, inplace=True)
cleaning8['location'].replace({'NEAR WINTER PAR, CO': 'Winter Park, Co'}, inplace=True)
cleaning8['location'].replace({'NEAR BORNITE, AK': 'BORNITE, AK'}, inplace=True)
cleaning8['location'].replace({'35 W. OF KETCHI, AK': 'KECHI, AK'}, inplace=True)
cleaning8['location'].replace({'10 SW OF MILAN, MO': 'MILAN, MO'}, inplace=True)
cleaning8['location'].replace({'GULF OF MEXICO': 'Mexico, MEXICO'}, inplace=True)
cleaning8['location'].replace({'STEAMBOAT SPRIN, CO': 'STEAMBOAT SPRINGS, CO'}, inplace=True)
cleaning8['location'].replace({'NEAR MERTZON, TX': 'MERTZON, TX'}, inplace=True)
cleaning8['location'].replace({'2NM NNM REDMOND, WA': 'REDMOND, WA'}, inplace=True)
cleaning8['location'].replace({'WEST CAMERON #4': 'WEST CAMERON, united states'}, inplace=True)
cleaning8['location'].replace({'MARMATH, ND': 'MARMARTH, ND'}, inplace=True)
cleaning8['location'].replace({'OSAGE BEECH, MO': 'OSAGE BEACH, MO'}, inplace=True)
cleaning8['location'].replace({'6NM SSE WRANGEL, AK': 'WRANGELL, AK'}, inplace=True)
cleaning8['location'].replace({'VERMAL, UT': 'VERNAL, UT'}, inplace=True)
cleaning8['location'].replace({'HUNTINGTON BEAC, CA': 'HUNTINGTON BEACH, CA'}, inplace=True)
cleaning8['location'].replace({'CHRISTMAS VALLE, OR': 'CHRISTMAS VALLEY, OR'}, inplace=True)
cleaning8['location'].replace({'INTRACOASTAL CI, LA': 'INTRACOASTAL CITY, LA'}, inplace=True)
cleaning8['location'].replace({'NEAR SAVOONGA, AK': 'SAVOONGA, AK'}, inplace=True)
cleaning8['location'].replace({'WEST DELTA 105D': 'WEST DELTA'}, inplace=True)
cleaning8['location'].replace({'NEAR KISSIMMEE, FL': 'KISSIMMEE, FL'}, inplace=True)
cleaning8['location'].replace({'NEAR TANANA, AK': 'TANANA, AK'}, inplace=True)
cleaning8['location'].replace({'NEAR TOUTLE, WA': 'TOUTLE, WA'}, inplace=True)
cleaning8['location'].replace({'NEAR BRIDEPORT, CA': 'BRIDGEPORT, CA'}, inplace=True)
cleaning8['location'].replace({'HIGH ISLAND BLK': 'HIGH ISLAND BLK, United States'}, inplace=True)
cleaning8['location'].replace({'GULF OF MEXICO': 'Mexico'}, inplace=True)
cleaning8['location'].replace({'10MI. NW WALSEN, CO': 'WALSEN, CO'}, inplace=True)
cleaning8['location'].replace({'NEAR KINGSTON, ID': 'KINGSTON, ID'}, inplace=True)
cleaning8['location'].replace({'S.E. OF MALTA, ID': 'MALTA, ID'}, inplace=True)
cleaning8['location'].replace({'REDHILL AERODRO, United Kingdom': 'REDHILL AERODROME'}, inplace=True)
cleaning8['location'].replace({'NORTH SEA, Netherlands': 'den helder'}, inplace=True)
cleaning8['location'].replace({'WENZHO, China': 'WENZHOU'}, inplace=True)
cleaning8['location'].replace({'11MI. SO.OFTELL, CO': 'TELL city, CO'}, inplace=True)
cleaning8['location'].replace({'NEAR RANDLE, WA': 'RANDLE, WA'}, inplace=True)
cleaning8['location'].replace({'INTERCOASTAL CI, LA': 'INTRACOASTAL CITY, LA'}, inplace=True)
cleaning8['location'].replace({'NEAR KALISPELL, MT': 'KALISPELL, MT'}, inplace=True)
cleaning8['location'].replace({'5NM EAST OF SAL, CA': 'SAL, CA'}, inplace=True)
cleaning8['location'].replace({'NEAR KEMMERER, WY': 'KEMMERER, WY'}, inplace=True)
cleaning8['location'].replace({'NEAR BROOTEN, MN': 'BROOTEN, MN'}, inplace=True)
cleaning8['location'].replace({'WEST CAMERON, united states': 'WEST CAMERON'}, inplace=True)


In [ ]:
cleaning8.iloc[2664, cleaning8.columns.get_loc('latitude')] = '27.5406'
cleaning8.iloc[2664, cleaning8.columns.get_loc('longitude')] = '-93.3443'
cleaning8.iloc[2661, cleaning8.columns.get_loc('country')] = 'Mexico'
cleaning8.iloc[2664, cleaning8.columns.get_loc('country')] = 'Mexico'
cleaning8.iloc[2730, cleaning8.columns.get_loc('country')] = 'Mexico'
cleaning8.iloc[2755, cleaning8.columns.get_loc('country')] = 'United States'
cleaning8.iloc[2765, cleaning8.columns.get_loc('country')] = 'Mexico'
cleaning8.iloc[2522, cleaning8.columns.get_loc('country')] = 'Mexico'
cleaning8.iloc[2523, cleaning8.columns.get_loc('country')] = 'Mexico'
cleaning8.iloc[2536, cleaning8.columns.get_loc('country')] = 'Mexico'
cleaning8.iloc[2611, cleaning8.columns.get_loc('country')] = 'Mexico'
cleaning8.iloc[2658, cleaning8.columns.get_loc('country')] = 'United States'


In [ ]:
list(cleaning9['model'].unique())

In [ ]:
cleaning9['engine_type_binary'] = cleaning9.engine_type1.map(lambda x: 1 if x == 'turbine' else 0)


In [ ]:
# Looking at what 'actual' values I have for weather. Consider breaking this off into a 'weather' df with dropping
# UNK and Nans (348 Nans as well)
cleaning9.weather_condition.value_counts()

In [ ]:
cleaning9.groupby('total_fatal_injuries')['engine_type1'].value_counts()


In [ ]:
# Can I use far_description to help populate purpose of flight? 
# Not able to make any generalisations from these categories.
x=cleaning6.groupby('far_description')['purpose_of_flight'].value_counts()
pd.DataFrame(x)

In [ ]:
cleaning_complete.to_csv('/Users/bronwyn/desktop/GA_Lessons/DSI8-lessons/projects/project-capstone/cleaning_complete_without_index.csv', index=False)
